## Visualisation tools

In [ ]:
from aimapp.visualisation_tools import plot_likelihood
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns
from matplotlib import colors


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def plot_transitions(B: np.ndarray, state_map: dict, actions: dict) -> np.ndarray:
    """Plot Transitions matrix showing the probability of a transition between two states given a certain action."""
    
    sorted_state_map = dict(sorted(state_map.items(), key=lambda item: item[1]['state']))
    labels = [f"{key} ({value['state']})" for key, value in sorted_state_map.items()]

    n_actions = len(actions)
    l = int(np.ceil(np.sqrt(n_actions)))
    L = int(np.ceil(n_actions / l))
    
    fig, axes = plt.subplots(L, l, figsize=(L*3 + max(10, 2.5*len(state_map)), 
                                             l*2 + max(10, 1.5*len(state_map))))
    
    axes = np.atleast_2d(axes)  # Ensure axes is always a 2D array
    count = 0

    for i in range(L):
        for j in range(l):
            if count >= n_actions:
                fig.delaxes(axes[i][j])
                continue
            
            if count not in actions:
                continue

            action_str = str(actions[count])  # Convert action name to string

            # Plot the heatmap
            g = sns.heatmap(B[:len(labels), :len(labels), count], cmap="OrRd", linewidth=3, 
                            cbar=False, ax=axes[i, j], xticklabels=labels, yticklabels=labels)

            g.tick_params(axis='both', which='major', labelsize=14)  # Adjust label font size
            g.set_title(action_str, fontsize=20)
            g.set_xlabel('Prev State', fontsize=16)
            g.set_ylabel('Next State', fontsize=16)

            # Rotate labels for better visibility
            g.set_xticklabels(labels, rotation=45, ha="right", fontsize=12)
            g.set_yticklabels(labels, rotation=0, fontsize=12)
            
            count += 1

    plt.subplots_adjust(left=0.2, bottom=0.2)  # Add margin space
    plt.tight_layout()

    return fig


In [3]:
def compare_B1_B2_plots(B1: np.ndarray, B2: np.ndarray, state_map: dict, actions: dict) -> np.ndarray:
    """Plot Transitions matrix showing the probability of a transition between two states given a certain action.
       Common values in B1 and B2 (within a margin of 0.1) are set to 0 in the resulting B.
    """
    
    # Create B by zeroing out common values within margin 0.1
    margin = 0.1
    B = np.where(np.abs(B1 - B2) <= margin,0, B2)
    
    sorted_state_map = dict(sorted(state_map.items(), key=lambda item: item[1]['state']))
    labels = [f"{key} ({value['state']})" for key, value in sorted_state_map.items()]

    n_actions = len(actions)
    l = int(np.ceil(np.sqrt(n_actions)))
    L = int(np.ceil(n_actions / l))
    
    fig, axes = plt.subplots(L, l, figsize=(L*3 + max(10, 2.5*len(state_map)), 
                                             l*2 + max(10, 1.5*len(state_map))))
    
    axes = np.atleast_2d(axes)  # Ensure axes is always a 2D array
    count = 0

    for i in range(L):
        for j in range(l):
            if count >= n_actions:
                fig.delaxes(axes[i][j])
                continue
            
            if count not in actions:
                continue

            action_str = str(actions[count])  # Convert action name to string

            # Plot the heatmap
            g = sns.heatmap(B[:len(labels), :len(labels), count], cmap="OrRd", linewidth=3, 
                            cbar=False, ax=axes[i, j], xticklabels=labels, yticklabels=labels)

            g.tick_params(axis='both', which='major', labelsize=14)  # Adjust label font size
            g.set_title(action_str, fontsize=20)
            g.set_xlabel('Prev State', fontsize=16)
            g.set_ylabel('Next State', fontsize=16)

            # Rotate labels for better visibility
            g.set_xticklabels(labels, rotation=45, ha="right", fontsize=12)
            g.set_yticklabels(labels, rotation=0, fontsize=12)
            
            count += 1

    plt.subplots_adjust(left=0.2, bottom=0.2)  # Add margin space
    plt.tight_layout()

    return fig, B

In [4]:
def plot_state_in_map(B: np.ndarray, state_mapping: dict,fig_ax=[None, None]) -> np.ndarray:
    """
    Plot states as dots positioned based on `state_mapping` keys.
    Draw transitions between states based on transition probabilities in `B`.

    Parameters:
    - B (np.ndarray): Transition matrix of shape (num_states, num_states, num_actions).
    - state_mapping (dict): Mapping of (x, y) positions to state properties.
    - possible_actions (dict): Dictionary of action indices to angle ranges.
    - pose_dist (float): Distance associated with each move action.

    Returns:
    - fig (matplotlib Figure): The generated figure.
    """
    if fig_ax[0] is None:
        fig, ax = plt.subplots(figsize=(25, 25))
    else:
        fig = fig_ax[0]
        ax = fig_ax[1]


    # Get unique observation values for color mapping
    unique_obs = np.sort(list({v['ob'] for v in state_mapping.values()}))
    color_map = get_cmap() #get_cmap('viridis', len(unique_obs))
    ob_to_color = {ob: color_map.colors[i] for i, ob in enumerate(unique_obs)}

    # Draw transitions between states
    num_states, _, num_actions = B.shape
    for prev_state in range(num_states):
        for next_state in range(num_states):
            for action in range(num_actions):
                prob = B[next_state, prev_state, action]
                if prob > 0.1:  # Only plot meaningful transitions
                    # Find corresponding positions in `state_mapping`
                    prev_pos = next((pos for pos, data in state_mapping.items() if data['state'] == prev_state), None)
                    next_pos = next((pos for pos, data in state_mapping.items() if data['state'] == next_state), None)
                    
                    if prev_pos and next_pos:
                        ax.plot([prev_pos[1], next_pos[1]], [prev_pos[0], next_pos[0]], 
                                'k-', linewidth=prob * 10)  # Scale linewidth with probability

    # Plot states as dots
    for (x, y), data in state_mapping.items():
        state = data['state']
        ob = data.get('ob', 0)
        color = ob_to_color[ob]

        ax.plot(y, x, 'o', color=color, markersize=20)  # Position state as (y, x)
        ax.text(y - 0.05, x + 0.05, str(state), fontsize=25, ha='right', c='r')  # Label state number

    # Formatting
    # ax.invert_yaxis()
    ax.invert_xaxis()
    ax.set_aspect('equal')
    ax.tick_params(axis='both', which='major', labelsize=26)
    plt.ylabel('X', fontsize=30)
    plt.xlabel('Y', fontsize=30)
    plt.title('State Transitions', fontsize=35)
    plt.grid(False)
    
    return fig
def create_custom_cmap(custom_colors) -> colors.ListedColormap:
    return colors.ListedColormap(custom_colors[:]) #,  alpha=None)

def get_cmap() -> colors.ListedColormap:
    custom_colors = (
            np.array(
                [
                    [255, 255, 255],#white 1
                    [255, 0, 0],#red 2
                    [0, 255, 0], #green 3
                    [50,50, 255], #bluish 4
                    [112, 39, 195], #purple5
                    [255, 255, 0], #yellow6
                    [100, 100, 100], #grey7
                    [115, 60, 60], #brown8
                    [255, 0, 255], #flash pink9
                    [80, 145,80], #kaki10
                    [201,132,226], #pink11
                    [75,182,220], #turquoise12
                    [255,153,51], #orange13
                    [255,204,229], #light pink14
                    [153,153,0], #ugly kaki 15
                    [229,255,204], #light green16
                    [204,204,255],#light purple17
                    [0, 153,153], #dark turquoise18
                    [138, 108, 106], #light brown19
                    [108, 115, 92],#ugly green20
                    [149, 199, 152],#pale green21
                    [89, 235, 210], #flashy light blue22
                    [37, 105, 122], #dark blue23
                    [22, 25, 92], #dark purple-blue24
                    [131, 24, 219], #flashy purple25
                    [109, 11, 120], #purple-pink26
                    [196, 145, 191], #pale pink27
                    [148, 89, 130], #dark pink28
                    [201, 75, 119], #pink-red29
                    [189, 89, 92], #light red30

                ]
            )
            / 256
        )

    n_colors = len(custom_colors)
    return create_custom_cmap(custom_colors[:n_colors])


## Model 

In [5]:
from aimapp.model.pymdp.agent import Agent
from aimapp.model.pymdp import utils
from aimapp.model.modules import *
from aimapp.model.pymdp.learning import update_obs_likelihood_dirichlet
from aimapp.model.odometry import PoseOdometry

# from aimapp.model.MCTS import MCTS
from aimapp.model.pymdp.control import get_expected_obs, get_expected_states, calc_states_info_gain, calc_expected_utility, calc_pA_info_gain, calc_pB_info_gain

from aimapp.model.pymdp.maths import softmax, spm_log_single

In [6]:
import random
import logging
import math

# --- Node Class ---
class Node:
    """
    Represents a node in the MCTS tree.
    Stores state information, MCTS statistics, and tree structure links.
    """
    def __init__(self, state_qs:np.ndarray, pose_id:int, parent:object=None, action_index:int=0, observation:np.ndarray=None, initial_reward:float=0.0, possible_actions=None):
        self.pose_id = pose_id
        self.id = pose_id  # Using pose_id as a unique identifier for the node

        # State Representation
        self.state_qs = state_qs  # Belief over states (e.g., particle filter weights)
        self.observation = observation # Observation associated with reaching this state (qo_pi)

        # MCTS Statistics
        self.total_reward = 0 #initial_reward # Sum of rewards accumulated through this node (Formerly T)
        self.N = 0 # Visit count

        # Tree Structure
        self.parent = parent # Reference to the parent node
        self.childs = {} # Dictionary mapping action -> child Node
        self.action_index = action_index # Action taken by the parent to reach this node

        # Action Space
        self.possible_actions = possible_actions # List of possible actions from this node's state (computed during expansion)
        self.untried_actions = None # Actions not yet explored from this node

        # Intrinsic Reward (EFE components calculated when node is evaluated)
        self.state_reward = initial_reward # The immediate EFE/G calculated for reaching this state

    def get_averaged_reward(self)->float:
        """Calculates the average reward accumulated through this node."""
        if self.N == 0:
            return 0.0 # Avoid division by zero for unvisited nodes
        return self.total_reward / self.N

    def get_ucb1_score(self, c_param:float=1.41)->float:
        """
        Calculates the UCB1 score for this node.
        Balances exploitation (average reward) and exploration (visit count).

        UCB1 ensures that the search doesn't prematurely focus only on the initially best-looking option but also invests simulations in exploring other potentially promising, but less certain, branches. This leads to more robust and accurate value estimates for the actions at the root node over many simulations.
        """
        if self.N == 0:
            return float('inf') # Prioritize exploring unvisited nodes

        if self.parent is None:
             # Should not happen during selection if root is handled correctly, but added for safety
             parent_visits = self.N
        else:
             parent_visits = self.parent.N

        if parent_visits == 0: # Avoid log(0) or division by zero if parent somehow has 0 visits
            parent_visits = 1

        exploitation_term = self.get_averaged_reward()
        exploration_term = c_param * math.sqrt(math.log(parent_visits) / self.N)

        # logging.debug(f"Node {self.id}: AvgReward={exploitation_term:.3f}, ExploitTerm={exploration_term:.3f}, ParentN={parent_visits}, SelfN={self.N}")

        # Add intrinsic state reward to bias selection towards immediately rewarding states
        # Note: Depending on the scale of state_reward vs rollout_reward, this might need tuning.
        return exploitation_term + exploration_term #+ self.state_reward

    def is_fully_expanded(self)->bool:
        """Checks if we verified possible actions from this node leading to a child node. This suppose we have No isolated node"""
        return self.possible_actions is not None and len(self.childs) > 0

    def has_children_nodes(self)->bool:
        """Checks if the node has any child nodes."""
        return len(self.childs) > 0
    
    def select_best_child_UCB(self, c_param:float=1.41)->object:
        """Selects the child with the highest UCB1 score."""
        best_score = -float('inf')
        best_child = None
        for action, child in self.childs.items():
            score = child.get_ucb1_score(c_param)
            logging.debug(f"  Child {child.id} (Action {action}) UCB1: {score:.2f}")
            
            if score > best_score:
                best_score = score
                best_child = child
        #logging.debug(f"Node {self.id}: Selected child {best_child.id if best_child else 'None'} with score {best_score:.2f}")
        return best_child
    
    def all_children_AIF(self)->list:
        """Selects the child with AIF."""
        all_averaged_efe = [c.get_averaged_reward() for c in self.childs.values()]
        # q_pi, best_action_id = self.infer_policy_over_actions(all_averaged_efe, self.possible_actions)
        # logging.debug(f"  Child {child.id} (Action {action}) average_EFE: {score:.2f}")    
        #logging.debug(f"Node {self.id}: Selected child {best_child.id if best_child else 'None'} with score {best_score:.2f}")
        return all_averaged_efe

    def detach_parent(self)-> None:
        """Removes the reference to the parent node to allow garbage collection."""
        logging.debug(f"Detaching parent from node {self.id}")
        del self.parent
        self.parent = None


# --- Model Interface Class ---
class MCTS_Model_Interface:
    """
    Acts as a wrapper or interface to the underlying Active Inference model.
    Provides methods to query the model for transitions, observations, rewards, etc.
    """
    def __init__(self, underlying_model:object):
        self.model = underlying_model # The actual model object (e.g., Ours_V5_RW instance)
        # Caches can be added here if needed for expensive model calls
        # self.transition_cache = {}
        # self.observation_cache = {}
        # self.reward_cache = {}
        logging.info(f"MCTS_Model_Interface initialized with model type: {type(underlying_model)}")

    def get_possible_actions(self)->list:
        """Returns a list of all possible actions [action_id]"""
        return list(self.model.get_possible_actions().keys())

    def id_to_pose(self, pose_id:int)->list:
        return self.model.PoseMemory.id_to_pose(pose_id)
    
    def get_next_node_pose_id(self, current_pose_id:int, action:int)->int:
        """Calculates the next pose ID resulting from taking an action."""   
        odom = self.model.PoseMemory.id_to_pose(current_pose_id)
        next_pose = self.model.PoseMemory.pose_transition_from_action(action, odom=odom)
        next_pose_id = self.model.PoseMemory.pose_to_id(next_pose, save_in_memory=False)
        next_pose = self.model.PoseMemory.id_to_pose(next_pose_id)
        pose_in_action_range = self.model.PoseMemory.pose_in_action_range(action, next_pose, odom= odom) #if we don't reach that pose with that action, we pass
        
        if not pose_in_action_range:
            #logging.warning(f"Action {action} from pose {current_pose_id} leads to unreachable pose {next_pose_id}. Invalid transition.")
            return -1 # Indicate invalid transition
        #logging.info(f"Action {action} from pose {odom}, {current_pose_id} leads to pose {next_pose}, {next_pose_id}. Invalid transition.")
        return next_pose_id

    def get_next_state_belief(self, current_belief_qs:np.ndarray, action:int)->np.ndarray:
        """Predicts the next belief state (qs) given the current belief and action."""
        # This corresponds to the belief state transition model p(qs'|qs, a)
        return self.model.get_next_state_given_action(qs=current_belief_qs, action=action)

    def get_expected_observation(self, next_belief_qs:np.ndarray)->np.ndarray:
        """Calculates the expected observation (qo_pi) given a belief state."""
        # This corresponds to p(o|qs')
        return self.model.get_expected_observation(next_belief_qs)

    def calculate_expected_free_energy(self, next_belief_qs:np.ndarray, expected_observation_qo_pi:np.ndarray, current_qs:np.ndarray, action:int)->float:
        """
        Calculates the Expected Free Energy (G) for a potential next state.
        G = Utility + Information Gain
        """
        G = 0.0
        logging.debug(f"action:{action}, next_belief_qs: {str(next_belief_qs)}")
        if self.model.use_states_info_gain:
            #the highest (>0), the more interesting
            info_gain = self.model.infer_info_gain_term([next_belief_qs]) # Assuming takes a list
            G += info_gain

            logging.debug(f"  Info Gain Term: {info_gain:.4f}")
        if self.model.use_utility:
            #the lowest (<0), the more interesting
            logging.debug(f"  Utility Term exp ob: {str(expected_observation_qo_pi)}")
            utility = self.model.infer_utility_term(expected_observation_qo_pi)
            G -= utility
            logging.debug(f"  Utility Term: {utility:.4f}")
        if self.model.use_param_info_gain: #not good in asociation with the other terms
            #the highest (>0), the less interesting
            param_info_gain = self.model.infer_param_info_gain([next_belief_qs],expected_observation_qo_pi, current_qs, action)[0]/100
            G -= param_info_gain
            logging.debug(f"  Param info gain Term: {param_info_gain:.4f}")

        logging.debug(f"  Calculated G: {G:.4f}")
        return G

    def infer_policy_over_actions(self, action_values:list, available_actions:list, action_selection:str=None, alpha:float=None):
        """Infers a probability distribution (policy) over actions based on their values (e.g., EFE)."""
        # This likely involves a softmax function as in the original code's example
        q_pi, best_action_id = self.model.infer_best_action_given_actions(action_values, available_actions,action_selection, alpha)
        return q_pi, best_action_id

# --- MCTS Algorithm Class ---
class MCTS:
    """
    Implements the Monte Carlo Tree Search algorithm using an Active Inference model.
    """

    def __init__(self, AIF_model:object, c_param:float=1.41, num_simulation:int=25, max_rollout_depth:int=10):
        self.model_interface = MCTS_Model_Interface(AIF_model)
        self.c_param = c_param # Exploration parameter for UCB1
        self.num_simulation  = num_simulation # Number of MCTS simulations per planning step
        self.max_rollout_depth = max_rollout_depth # Maximum depth for the simulation (rollout) phase
        logging.info(f"MCTS initialized with exploration parameter c={c_param}, num_simus={num_simulation}, max_depth={max_rollout_depth}, policy_alpha={AIF_model.alpha},  action_selection={AIF_model.action_selection}")

    
    def start_mcts(self,state_qs:np.ndarray, pose_id:int, observation:np.ndarray, next_possible_actions:list= None, num_steps:int=1, logging=None)-> list:
        current_node = Node(state_qs=state_qs,
                pose_id=pose_id,
                parent=None,
                action_index=None,
                observation=observation, 
                possible_actions=next_possible_actions)
        
        best_actions = []
        data = {"qs": state_qs[0],
            "qpi": [],
            "efe": [],
            "info_gain": [],
            "utility": [],
            #"bayesian_surprise": utils.bayesian_surprise(posterior[0].copy(), prior),
            }
        for i in range(num_steps):
            best_action, data = self.plan(current_node, self.num_simulation, self.max_rollout_depth, data)
            best_actions.append(best_action)
            if num_steps>1 and best_action in current_node.childs:
                next_node = current_node.childs[best_action]
                if logging:
                    logging.info(f"Executing action {best_action} -> Transitioning to Node {next_node.id}")

                # IMPORTANT: Detach the chosen next state from its parent (the previous state).
                # This makes the chosen next state the new root for the *next* planning step
                # and allows the old parts of the tree to be garbage collected.
                next_node.detach_parent()
                current_node = next_node # Update the current state

        return best_actions, data

    def _select_node(self, root_node:object)->object:
        """Phase 1: Selection - Traverse the tree using UCB1 until a leaf node is reached."""
        current = root_node
        # logging.debug(f"--- Selection Phase Start (Root: {root_node.id}) ---")
        while current.is_fully_expanded():
            logging.debug(f"  Selected Node {current.id}")
            # logging.debug(f"Selecting from Node {current.id} (N={current.N}, TR={current.total_reward:.3f})")
            #USING UCB
            current = current.select_best_child_UCB(self.c_param)
            #USING AIF
            # children_G = current.all_children_AIF()
            # q_pi, best_action = self.model_interface.infer_policy_over_actions(children_G, current.possible_actions, action_selection='stochastic', alpha=1.0)
            # current = current.childs[best_action]

        logging.info(f"--- Selection Phase End (Selected Node: {current.id}) ---")
        return current

    def _expand_node_in_all_possible_direction(self, node:object)->object:
        """Phase 2: Expansion - Add a new child node for an untried action."""
        
        node.possible_actions =[]
        node.childs = {}
            
        all_possible_actions = self.model_interface.get_possible_actions()

        #we save as the current node child each new node created taking an action from current pose 
        for action in all_possible_actions:
            #=== check if new  ===#
            if action not in node.possible_actions:
                next_pose_id = self.model_interface.get_next_node_pose_id(node.pose_id, action)
                if next_pose_id < 0: #no known or valid next node
                    continue
                node.possible_actions.append(action)
            
            #=== action leading to a node, saving believed qs and qo ===#
            # print('node.state_qs', node.state_qs[0].round(4), action)
            next_state_qs = self.model_interface.get_next_state_belief(node.state_qs, action=action)[0]
            # print('next_state_qs', next_state_qs[0][0].round(3))
            qo_pi = self.model_interface.get_expected_observation(next_state_qs)
            #python should erase unreferenced classes. But let's systematise it
            if action in node.childs:
                del node.childs[action]
            # Calculate the immediate reward (Expected Free Energy) for this transition
            # Note: This G is associated with *reaching* the new state.
            child_reward_G = self.model_interface.calculate_expected_free_energy(next_state_qs, qo_pi, node.state_qs, action)

            # Create the new child node
            child_node = Node(
                state_qs=next_state_qs,
                pose_id=next_pose_id,
                parent=node,
                action_index=action,
                observation=qo_pi,
                initial_reward= child_reward_G 
                # Rollout will determine total_reward
                # possible_actions will be determined when the child is expanded later
            )

            #To get a headstart (not necessary)
            # child_node.N +=1
            # child_node.total_reward = child_node.state_reward 
            # parent = child_node
            # while parent.parent:
            #     parent = parent.parent
            #     parent.N += 1
            #     parent.total_reward = parent.total_reward + child_reward_G

            node.childs[action] = child_node
            logging.debug(f"from node {node.id} -> Child Node {child_node.id}, expanding with action {action}(Initial G={child_reward_G:.3f})")
            # logging.debug(f"--- Expansion Phase End (Expanded Node: {child_node.id}) ---")
        return node # Return the newly expanded node

    def _rollout(self, start_node:object, max_depth:int)->float:
        """
        Phase 3: Simulation (Rollout) - Simulate a trajectory from the start_node
        using a default policy (e.g., random actions) and return the cumulative reward (G).
        """
        # logging.debug(f"--- Rollout Phase Start (Node: {start_node.id}, Max Depth: {max_depth}) ---")
        current_sim_qs = start_node.state_qs
        current_sim_pose_id = start_node.pose_id
        cumulative_G = 0.0
        depth = 1

        current_node = start_node

        while depth < max_depth:
            # 1. Get possible actions from the *current simulated pose*
            if current_node and current_node.is_fully_expanded():
                all_possible_actions = current_node.possible_actions
            else:
                all_possible_actions = self.model_interface.get_possible_actions()
            
            if len(all_possible_actions)==0:
                # logging.debug(f"  Rollout Depth {depth}: No actions possible from pose {current_sim_pose_id}. Stopping.")
                break # Dead end in simulation

            # 2. Choose an action using the default policy (random)
            action = random.choice(all_possible_actions)

            # 3. Simulate the transition
            next_sim_pose_id = self.model_interface.get_next_node_pose_id(current_sim_pose_id, action)
            if next_sim_pose_id < 0:
                 # logging.debug(f"  Rollout Depth {depth}: Action {action} from pose {current_sim_pose_id} leads to invalid state. Stopping.")
                 break # Invalid move in simulation

            next_sim_qs = self.model_interface.get_next_state_belief(current_sim_qs, action)[0]
            sim_qo_pi = self.model_interface.get_expected_observation(next_sim_qs)

            # 4. Calculate reward (G) for this simulated step
            step_G = self.model_interface.calculate_expected_free_energy(next_sim_qs, sim_qo_pi, current_sim_qs, action)
            cumulative_G += step_G
            logging.debug(f"  Rollout Depth {depth}: Action {action}, NextPose {next_sim_pose_id}, StepG={step_G:.3f}, CumulG={cumulative_G:.3f}")

            # 5. Update simulated state
            current_sim_qs = next_sim_qs
            current_sim_pose_id = next_sim_pose_id
            depth += 1

            # 6. If a node exist for that action, retrieve it to get appropriate next actions
            if current_node and current_node.has_children_nodes():
                current_node = current_node.childs.get(action,None)
            else:
                current_node = None

        # logging.debug(f"--- Rollout Phase End (Node: {start_node.id}, Total Rollout G: {cumulative_G:.3f}) ---")
        return cumulative_G / depth
    
    def _backpropagate(self, node:object, reward:float)-> None:
        """Phase 4: Backpropagation - Update visit counts and total rewards up the tree."""
        # logging.debug(f"--- Backpropagation Start (Node: {node.id}, Reward: {reward:.3f}) ---")
        current = node
        while current is not None:
            current.N += 1
            current.total_reward += reward
            # logging.debug(f"  Updating Node {current.id}: N={current.N}, TR={current.total_reward:.3f}")
            current = current.parent
        # logging.debug(f"--- Backpropagation End ---")

    def run_simulation(self, root_node, max_rollout_depth):
        """Runs a single iteration of the MCTS algorithm (Select, Expand, Simulate, Backpropagate)."""
        # logging.debug(f"=== Starting MCTS Simulation ===")

        # Phase 1: Selection
        selected_node = self._select_node(root_node)

        # Phase 2: Expansion
        # If the selected node is not terminal and not fully expanded, expand it.
        # Check if the node is terminal (add domain-specific logic if needed, e.g., goal reached)
        # is_terminal = False # Placeholder - add condition if applicable
        # if not is_terminal:
        if not selected_node.is_fully_expanded():
            selected_node = self._expand_node_in_all_possible_direction(selected_node)
        else:
            # If fully expanded, the rollout starts from the selected node itself
            # This can happen if selection leads to an already expanded node
            logging.debug(f"Selected node {selected_node.id} is fully expanded, starting rollout from here.")
            #pass


        # Phase 3: Simulation (Rollout)
        # Start rollout from the newly expanded node (or the selected node if expansion wasn't possible/needed)
        reward = self._rollout(selected_node, max_rollout_depth)
        # Add the immediate state reward (G) of the node where the rollout started
        # This connects the immediate EFE gain with the future expected gains from the rollout
        reward += selected_node.state_reward

        # Phase 4: Backpropagation
        self._backpropagate(selected_node, reward)
        
        children_info = [('child id',c.id,'N',c.N,'T', round(c.total_reward,2),'efe_av', round(c.get_averaged_reward(),2)) for a,c in root_node.childs.items()]
        logging.info(f"Root node children stats: {children_info}")
        # logging.debug(f"=== Finished MCTS Simulation ===")

    def plan(self, root_node:object, num_simulations:int, max_rollout_depth:int, data:dict=None)-> int: #dict
        """Runs the MCTS planning process for a given number of simulations."""
        logging.info(f"Starting MCTS planning from root node {root_node.id} for {num_simulations} simulations.")
        for i in range(num_simulations):
            print()
            logging.info(f"--- Simulation {i+1}/{num_simulations} ---")
            self.run_simulation(root_node, max_rollout_depth)

        # After simulations, determine the best action from the root
        best_action, q_pi_actions_values = self.get_best_action(root_node)
        data['qpi'].append(q_pi_actions_values[0])
        data['efe'].append(q_pi_actions_values[1])
        logging.info(f"MCTS planning finished. Best action: {best_action}")
        return best_action, data

    def get_best_action(self, root_node:object)->int:
        """Selects the best action from the root node after simulations."""
        if not root_node.childs:
            logging.warning("Root node has no children after simulations. Cannot determine best action.")
            return None # Or a default action

        # Option 1: Choose the most visited child (robust)
        # best_action = max(root_node.childs.keys(), key=lambda action: root_node.childs[action].N)

        # Option 2: Choose the child with the highest average reward (can be greedy)
        # best_action = max(root_node.childs.keys(), key=lambda action: root_node.childs[action].get_averaged_reward())

        # Option 3: Use the model's policy inference based on average rewards (AIF scheme)
        action_values = []
        available_actions = []
        child_info = []
        for action, child in root_node.childs.items():
            avg_reward = child.get_averaged_reward()
            action_values.append(avg_reward)
            available_actions.append(action)
            child_info.append(f"Action {action}: AvgR={avg_reward:.3f}, N={child.N}")
        logging.info(f"Root node children stats: {'; '.join(child_info)}")

        if len(available_actions)==0:
             logging.warning("No valid actions available from root node children.")
             return None, []

        q_pi, best_action_id = self.model_interface.infer_policy_over_actions(action_values, available_actions)
        logging.info(f"action average G: {action_values}")
        logging.info(f"softmax policies: {q_pi.round(2)}")
        logging.info(f"Selected best action based on policy: {best_action_id}")
        
        # Ensure the selected action is actually one of the children
        if best_action_id not in root_node.childs:
             logging.error(f"Policy selected action {best_action_id} which is not a child of the root node. Available: {list(root_node.childs.keys())}. Falling back to most visited.")
             # Fallback to most visited
             if available_actions:
                best_action_id = max(root_node.childs.keys(), key=lambda action: root_node.childs.get(action).N if root_node.childs.get(action) else -1)
             else:
                 return None, [] # No valid children

        full_action_values = [action_values[available_actions.index(a)] if a in available_actions else 0 for a in self.model_interface.get_possible_actions()]
        full_q_pi = [q_pi[available_actions.index(a)] if a in available_actions else 0 for a in self.model_interface.get_possible_actions()]
        return best_action_id, (full_q_pi, full_action_values)

In [7]:
class Ours_V5_RW(Agent):
    #====== NECESSARY TO SETUP MODEL ======#
    def __init__(self, num_obs=2, num_states=2, dim=2, observations=[0,(0,0)], lookahead_policy=4, \
                 learning_rate_pB=3.0, n_actions= 6,\
                 influence_radius:float=0.5, robot_dim:float=0.25, \
                   lookahead_node_creation=3) -> None:
        self.agent_state_mapping = {} #testing purposes
        self.influence_radius = influence_radius
        self.robot_dim = robot_dim 
        self.possible_actions = self.generate_actions(n_actions) 
        self.PoseMemory = PoseOdometry(self.possible_actions, influence_radius, robot_dim)

        self.preferred_ob = [-1,-1]
        self.simple_paths = True # less computationally expensive path than full coverage paths 
                                 #(consider only a few direction and never go back on path)
        self.lookahead_node_creation = lookahead_node_creation
        observations, agent_params = self.create_agent_params(num_obs=num_obs, num_states=num_states, observations=observations, \
                            learning_rate_pB=learning_rate_pB, dim=dim, lookahead_policy=lookahead_policy)
        super().__init__(**agent_params)
        self.initialisation(observations=observations)
    
    def create_agent_params(self,num_obs:int=2, num_states:int=2, observations:list=[0,(0,0)], 
                    learning_rate_pB:float=3.0, dim:int=2, lookahead_policy:int=4):
        ob = observations[0]
        p_idx = -1
        if dim > 1:
            #start pose in map
            if len(observations) < 2:
                observations.append([0.0,0.0])
            self.PoseMemory.reset_odom(observations[1])
            p_idx = self.PoseMemory.pose_to_id()
            observations[1] = p_idx
            
        else:
            p_idx = self.PoseMemory.pose_to_id()
        
        self.current_pose = self.PoseMemory.get_odom(as_tuple=True)
        #INITIALISE AGENT PARAMS
        B_agent = create_B_matrix(num_states,len(self.possible_actions))
        if 'STAY' in self.possible_actions and self.set_stationary_B:
            B_agent = set_stationary(B_agent,self.possible_actions['STAY'])
        pB = utils.to_obj_array(B_agent)

        obs_dim = [np.max([num_obs, ob + 1])] + ([np.max([num_obs, p_idx + 1])] if dim > 1 else [])
        A_agent = create_A_matrix(obs_dim,[num_states]*dim,dim)
        pA = utils.dirichlet_like(A_agent, scale = 1)

        return observations, {
            'A': A_agent,
            'B': B_agent,
            'pA': pA,
            'pB': pB,
            'policy_len': lookahead_policy,
            'inference_horizon': lookahead_policy,
            'lr_pB': learning_rate_pB,
            'lr_pA': 5,
            'save_belief_hist': True,
            'action_selection': "stochastic", 
            'use_param_info_gain': False
        }

    def initialisation(self,observations:list=[0,[0,0]]):
        """
        Initialises the agent with the first observation and ensures all parameters 
        are suitable for continuous navigation.

        Parameters:
            observations (list, optional): Initial observation. Defaults to [0, [0, 0]].
            linear_policies (bool, optional): 
                - If **False**: Explores all combinations of actions (exponential complexity: `n_action^policy_len` with `policy_len == lookahead`).
                - If **True**: Generates a linear path reaching a **lookahead distance** or **num steps**.
                - The path remains linear if no "STAY" actions are included.
                - If "STAY" actions exist, the path follows a polynomial pattern.
                - "STAY" actions are irregular and appear only at the end of a policy.
            E (optional): Additional environment-specific parameters (default: None).

        Note:
            - `linear_policies=True` is optimized for cases where `num_factor == 1` 
            and `len(num_control) == 1`.

        Returns:
            None
        """
      
        # self.init_policies()
        self.reset(start_pose=self.PoseMemory.get_poses_from_memory()[0])
        if self.edge_handling_params["use_BMA"] and hasattr(self, "q_pi_hist"): #This is not compatible with our way of moving
            del self.q_pi_hist
            
        self.inference_params = {'num_iter': 3, 'dF': 1.0, 'dF_tol': 0.001}
        #Not necessary, but cleaner
        for i in range(len(self.A)):
            self.A[i][:,:] = 0.01 #reset A for cleaner plot and more fair state inference
        self.update_A_with_data(observations,0)
        self.update_agent_state_mapping(self.current_pose, observations, 0)
        self.infer_states(observation = observations, partial_ob=None)
        if 'STAY' in self.possible_actions.values():
            stay_action = [key for key, value in self.possible_actions.items() if value == 'STAY'][0]
            self.B[0] = set_stationary(self.B[0], stay_action)
        return 
    
    def init_policies(self):
        policies = create_policies(self.policy_len, self.possible_actions,simple_paths= self.simple_paths)
        self.policies = policies
        assert all([len(self.num_controls) == policy.shape[1] for policy in self.policies]), "Number of control states is not consistent with policy dimensionalities"
        
        all_policies = np.vstack(self.policies)

        assert all([n_c >= max_action for (n_c, max_action) in zip(self.num_controls, list(np.max(all_policies, axis =0)+1))]), "Maximum number of actions is not consistent with `num_controls`"
        # Construct prior over policies (uniform if not specified) 
        self.E = self._construct_E_prior()

    def reset(self, init_qs:np.ndarray=None, start_pose:tuple=None):
        """
        Resets the agent's posterior beliefs about hidden states to a uniform distribution 
        and resets the simulation time to the initial timestep.

        This function initializes or resets key agent parameters, including past actions, 
        observations, and beliefs, ensuring proper inference and navigation behavior.

        Parameters
        ----------
        init_qs : numpy.ndarray, optional
            A predefined posterior over hidden states. If provided, the agent's beliefs 
            will be initialized using `init_qs` instead of a uniform prior.
        
        start_pose : tuple, optional
            The initial position (pose) of the agent. If provided, it sets `self.current_pose`.

        Returns
        -------
        qs : numpy.ndarray
            The initialized posterior over hidden states. The structure of `qs` depends on 
            the inference algorithm selected:

            - If `self.inference_algo == 'VANILLA'`:  
            `qs` is a simple uniform distribution over hidden states.

        Notes
        -----
        - If `self.edge_handling_params['policy_sep_prior']` is enabled, 
        the latest beliefs are initialized separately for each policy.
        - If `init_qs` is provided, it is directly assigned to `self.qs`, 
        bypassing uniform initialization.
        """

        self.curr_timestep = 0
        self.action = None
        self.prev_actions = None
        self.prev_obs = []
        self.qs_step = 0
     
        self.current_pose = start_pose
        if init_qs is None:
            
            self.D = self._construct_D_prior()
           
            if hasattr(self, "q_pi_hist"):
                self.q_pi_hist = []

            if hasattr(self, "qs_hist"):
                self.qs_hist = []
            
            if self.inference_algo == 'VANILLA':
                self.qs = utils.obj_array_uniform(self.num_states)
            else:
                print('MMP INFERENCE NOT IMPLEMENTED')
        
        else:
            self.qs = init_qs

        return self.qs
    
    def generate_actions(self,n_actions:int)->dict:
        """
        Divides the 360-degree orientation into discrete action zones and 
        returns a dictionary mapping each action to its corresponding range.

        Parameters:
            n_actions (int): The number of discrete actions to divide the 
                            360-degree space into.

        Returns:
            dict: A dictionary where keys are action indices (int), and values 
                are lists containing the start and end zone (in degrees):
                `{action_index: [start_zone, end_zone]}`.

        Note:
            - The action zones are evenly spaced across 360 degrees.
            - The function include a "STAY" action.
            - The start and end values are rounded to the nearest integer.
        """
        stay = False
        if n_actions% 2 != 0:
            n_actions = n_actions-1
            stay = True
        zone_range_deg = round(360/n_actions,1)
        n_actions_keys = np.arange(0, n_actions, 1)
        zone_spacing_deg = np.arange(0, 361, zone_range_deg)
        possible_actions = {}
        for action_key in n_actions_keys:
            possible_actions[action_key] = [round(zone_spacing_deg[action_key]), round(zone_spacing_deg[action_key+1]),]
        if stay:
            possible_actions[len(possible_actions)] = "STAY"

        return possible_actions


    #==== TEST&VISU PURPOSES ONLY ====#
    def update_agent_state_mapping(self, pose:tuple, ob:list, state_belief:list=None)-> dict:
        """ Dictionnary to keep track of believes and associated obs, usefull for testing purposes"""
        if state_belief is None:
            state = -1
        else:
            state = np.argmax(state_belief)
        #If we already have an ob, let's not squish it with ghost nodes updates
        if pose in self.agent_state_mapping.keys() and self.agent_state_mapping[pose]['ob'] != -1:
            ob[0] = self.agent_state_mapping[pose]['ob']
        self.agent_state_mapping[pose] = {'state' : state , 'ob': ob[0]}
        if len(ob) > 1:
           self.agent_state_mapping[pose]['ob2'] =  ob[1] 
      
        return self.agent_state_mapping
    
    #==== GET METHODS
    def get_current_pose_id(self):
        ''' we do not want a negative pose od'''
        if self.current_pose is None:
            current_pose = self.PoseMemory.get_odom()[:2]
        else:
            current_pose = self.current_pose
        return self.PoseMemory.pose_to_id(current_pose)
    
    def get_belief_over_states(self):
        """
        get the belief distribution over states

        Returns:
            np.ndarray: The extracted belief distribution over states.
        """
        return self.qs
    
    def get_current_timestep(self):
        return self.curr_timestep
    def get_possible_actions(self):
        return self.possible_actions
    def set_memory_views(self, views):
        self.ViewMemory = views
    def get_memory_views(self):
        return self.ViewMemory
    
    def get_agent_state_mapping(self)->dict:
        return self.agent_state_mapping
    
    def get_B(self):
        return self.B[0]
    
    def get_A(self):
        return self.A
    def get_current_most_likely_pose(self, z_score:float, min_z_score:float=2, qs=None,  observations:list=[])->int:
        """
        Given a z_scores (usually around 2), is the agent certain about the state. If it is, to which pose does it correspond?
        Return pose -1 if < threhsold, else return pose id.
        If no state stands out at all, we don't know where we are and return -2
        """
        if qs is None:
            qs = self.get_belief_over_states()[0]
        p_idx = -1
        mean = np.mean(qs)
        std_dev = np.std(qs)
        #print('qs mean and std_dev', mean, std_dev)
        # Calculate Z-scores
        z_scores = (qs - mean) / std_dev
        # Get indices of values with Z-score above 2
        outlier_indices = np.where(np.abs(z_scores) > z_score)[0]
        min_outlier_indices = np.where(np.abs(z_scores) > min_z_score)[0]
        
        
        #print("Indices of outliers (Z-score >",z_score,"):" , outlier_indices)
        #If we are sure of a state (independent of number of states), we don't have pose as ob and A allows for pose
        if len(outlier_indices) >= 0 and len(observations) < 2 and len(self.A) > 1:
            #If 1 state stands out
            if len(outlier_indices) == 1:
                p_idx = outlier_indices[0]
        #If min_z_scores length is 0, it means no proba is standing out! We don't know where we are   
        elif len(min_outlier_indices) == 0 and len(observations) < 2 and len(self.A) > 1:
            p_idx = -2
        return p_idx

    def get_observation_most_likely_states(self, z_score:float, observations:list=[])->int:
        """
        Given a z_scores (usually around 2), is the agent certain about the state. If it is, to which pose does it correspond?
        Return pose -1 if < threhsold, else return pose id.
        If no state stands out at all, we don't know where we are and return -2
        """
        likelihood = self.get_A()[0][observations[0],:]
        p_idx = -1
        mean = np.mean(likelihood)
        std_dev = np.std(likelihood)
        print('likelihood mean and std_dev', mean, std_dev)
        # Calculate Z-scores
        z_scores = (likelihood - mean) / std_dev
        # Get indices of values with Z-score above 2
        outlier_indices = np.where(np.abs(z_scores) > z_score)[0]       
        
        print("likelihood Indices of outliers (Z-score >",z_score,"):" , outlier_indices)
        #If we are sure of a state (independent of number of states), we don't have pose as ob and A allows for pose
    
        return outlier_indices
    
    def get_expected_observation(self, qs=np.ndarray, A:np.ndarray=None)-> np.ndarray:
        """ get observation belief for state qs"""
        if A is None:
            A = self.A
        qo_pi = get_expected_obs(qs, A)
        return qo_pi

    def get_next_state_given_action(self, qs= np.array, action=int, B=None)->np.ndarray:
        ''' expect only 1 qs, return only 1 qs with the same shape (1,) == np.ndarray([np.ndarray([])])'''
        if B is None:
            B = self.B

        # print('B check', B[0][:,np.argmax(qs), action])
        if isinstance(action, (int,np.int64)):
            action = np.array([[action]])
            
        qs_pi = get_expected_states(qs, B, action)
        return qs_pi
    
    
    #==== MCTS_CALL ====#
    def define_actions_from_MCTS_run(self, logging=None, num_steps=1, **kwargs)->list: #,dict
        """ 
        MCTS RUN, UNDER TEST (SHOULD NOT BE RE-CREATED EACH RUN)
        TODO: adapt for when we want a full policy

        """
        num_simulations = 25  # Number of MCTS simulations per planning step
        max_rollout_depth = 7 # Maximum depth for the simulation (rollout) phase
        c_param = 5
        mcts = MCTS(self, c_param, num_simulations, max_rollout_depth)

        observations = kwargs.get('observations', None)
        next_possible_actions = kwargs.get('next_possible_actions', None)

        #If we are not inferring state at each state during the model update, we do it here
        if observations is not None and self.current_pose is None:
        #If self.current_pose is not None then we have step_update that infer state
            
            #NB: Only give obs if state not been inferred before 
            if len(observations) < len(self.A):
                partial_ob = 0
                            
            elif len(observations) == len(self.A):
                partial_ob = None
                if self.current_pose == None:
                    self.current_pose = observations[1]
                observations[1] = self.PoseMemory.pose_to_id(observations[1])
            
            self.infer_states(observation = observations, partial_ob=partial_ob, save_hist=True)

        initial_pose_id = self.get_current_pose_id()
        initial_belief_qs = self.get_belief_over_states() # Get initial belief
        initial_observation = self.get_expected_observation(initial_belief_qs)

        best_actions, data = mcts.start_mcts(state_qs=initial_belief_qs,
                     pose_id=initial_pose_id, observation=initial_observation, \
                     next_possible_actions=next_possible_actions, num_steps=num_steps, logging=None)
        
        #NOTE: THIS CONSIDERONLY FIRST ACTION OF POLICY. MIGHT LEADS TO ISSUE DEPENDING ON HOW WE USE THAT
        self.q_pi = data['qpi'][0]
        self.G = data['efe'][0]

        #NOTE: THIS CONSIDER THAT WE APPLY FIRST ACTION OF POLICY. MIGHT LEADS TO ISSUE DEPENDING ON HOW WE USE THAT
        # self.action = np.array([best_actions[0]])
        # self.step_time()
        
        return best_actions[:num_steps], data
    #==== INFERENCE ====#

    def infer_states(self, observation:list, action:np.ndarray= None ,save_hist:bool=True, partial_ob:int=None, qs:list=None):
        """
        Performs variational inference to update posterior beliefs over hidden states given an observation.

        This method updates the agent's belief state (`qs`) by incorporating new observations 
        and optionally considering the previous action. The update process depends on the 
        selected inference algorithm (`VANILLA`).

        Parameters
        ----------
        observation : list or tuple 
            The observed state indices for each observation modality.
    
        action : np.ndarray, optional
            The most recent action taken by the agent. If provided, it helps refine posterior beliefs.

        save_hist : bool, default=True
            If True, stores the latest observation and updates historical data for future inference.


        partial_ob : int, optional
            Specifies a particular observation modality to update the belief state for, rather than all modalities.

        qs : list, optional
            A predefined posterior belief state. If provided, this will be used instead of computing from scratch.

        Returns
        -------
        qs : numpy.ndarray of dtype object
            Updated posterior beliefs over hidden states. The structure depends on the inference algorithm:
            - For `VANILLA`, `qs` represents a single posterior belief over hidden states.


        Notes
        -----
        - If `self.inference_algo == "VANILLA"`, posterior updates consider an empirical prior derived from 
        the transition model (`B`) or from a uniform prior (`D`).
        - The method also updates `self.qs_hist` and `self.qs_step` when `save_hist=True`, 
        enabling tracking of belief evolution over time.
        """

        if save_hist:
            self.prev_obs.append(observation)
            observations_hist = self.prev_obs
        else:
            observations_hist = self.prev_obs.copy()
            observations_hist.append(observation)

        if action == None:
            action = self.action

        if self.inference_algo == "VANILLA":
            if action is not None:
                if qs is None:
                    ref_qs = self.get_belief_over_states() #we don't yet want to consider current obs to selest qs
                else: #safety to avoid any risk
                    ref_qs = qs[:]
                empirical_prior = control.get_expected_states(
                    ref_qs, self.B, action.reshape(1, -1) #type: ignore
                )[0]
            #unused, but kept as a memor
            else:
                self.D = self._construct_D_prior() #self.D
                empirical_prior = self.D
            if self.current_pose is None:
                #TODO: increase A with observation even when self.current_pose is None
                for i in range(len(self.A)):
                    if partial_ob != None:
                        i = partial_ob
                    if observation[i] >= len(self.A[i]):
                        print('ERROR IN INFER STATE: given observation not in A')
                        qs = self.get_belief_over_states()
                        return qs
            qs = update_posterior_states(
            self.A,
            observation,
            empirical_prior,
            partial_ob,
            **self.inference_params
            )
            F = 0
            qs_step = 0

        if save_hist:
            self.F = F # variational free energy of each policy  
            self.qs_step = qs_step
            if hasattr(self, "qs_hist"):
                self.qs_hist.append(qs)
            self.qs = qs

        return qs
    
    def infer_pose(self, pose)->list:
        '''
        Parameters:
            pose (int or list): The index of the pose or the pose itself
        Here we consider that action consider the actual action sensed by agent (thus consider no motion)
        and we consider PoseMemory adapted to treat that perception
        '''
        if isinstance(pose,int):
            pose = self.PoseMemory.id_to_pose(pose)
        self.PoseMemory.update_odom_given_pose(pose)
        if self.current_pose !=None:
            self.current_pose = self.PoseMemory.get_odom(as_tuple=True)[:2]
        return self.current_pose
    
    def infer_best_action_given_actions(self, G:list, actions:list, action_selection:str=None, alpha:float=None):
        if isinstance(actions[0], (int, np.int64)):
            actions = np.array([[[a]] for a in actions])

        if action_selection is None:
            action_selection = self.action_selection
        if alpha is None:
            alpha = self.alpha
        G = np.array(G)
        lnE = spm_log_single(np.ones(G.shape) / len(G))

        q_pi = softmax(G * self.gamma + lnE) 
        if self.sampling_mode == "marginal":
            best_action = control.sample_action(q_pi, policies = actions, num_controls = self.num_controls, action_selection = action_selection, alpha= alpha)
        elif self.sampling_mode == "full":
            best_action = control.sample_policy(q_pi, actions, self.num_controls,
                                           action_selection=action_selection, alpha=alpha)
        return q_pi, int(best_action[0])
    
    def infer_utility_term(self, qo_pi:np.ndarray, C=None)->float:
        """ given the observation belief of a state, what is the utility term"""
        if C is None:
            C = self.C
        return calc_expected_utility(qo_pi, C)
    
    def infer_info_gain_term(self, qs_pi:np.ndarray, A=None)->float:
        """ given the belief of a state, what is the info gain term (Note the method expects several qs, thus qs must be in 3 layers of np.ndarray)"""
        if A is None:
            A = self.A
        return calc_states_info_gain(A, qs_pi)
    
    def infer_param_info_gain(self, qs_pi:np.ndarray, qo_pi:np.ndarray, qs:np.ndarray, action:int):
        """Infer param info gain for an action (but can also be a policy)"""
        G = 0
        if self.pA is not None:
            param_info_gain = calc_pA_info_gain(self.pA, qo_pi, qs_pi)
            G +=  param_info_gain
        if self.pB is not None:
            if isinstance(action, (int,np.int64)):
                action = np.array([[action]])
            param_info_gain = calc_pB_info_gain(self.pB, qs_pi, qs, action)
            G +=  param_info_gain

        return G
    #==== OTHER METHODS ====#

    def determine_next_pose(self, pose, action_id, min_dist_to_next_node):
        next_pose = self.PoseMemory.pose_transition_from_action(action =action_id, odom= pose, ideal_dist=min_dist_to_next_node)
        next_pose = [round(elem, 2) for elem in next_pose]
        next_pose_id = self.PoseMemory.pose_to_id(next_pose, save_in_memory=False)
        print('action, next pose and id', action_id, next_pose, next_pose_id)
        return next_pose, next_pose_id

    def determine_action_given_angle_deg(self, angle):
        """
        Find the key in possible_actions corresponding to the given angle.

        Args:
            angle (float): The angle to check in DEGREES

        Returns:
            int: The corresponding action key, or None if no match is found.
        """
        actions = self.possible_actions.copy()
        if "STAY" in actions.values():
            actions.popitem()
        action_key = [k for k,v in actions.items() if v[0] <= angle and v[1] > angle]
        return action_key[0]
        
        #same thing
        # for key, value in self.possible_actions.items():
        #     if value == "STAY":
        #         continue
        #     if value[0] <= angle < value[1]:  # Check if angle falls within range
        #         return key
        # return None

    def agent_manual_action_step(self, action):
        self.action = np.array([action])
        self.step_time()
    
    #==== Update A, B, C ====#
    def update_A_with_data(self,obs:list, state:int)->np.ndarray:
        """Given obs and state, update A entry """
        A = self.A
        
        for dim in range(self.num_modalities ):
            A[dim][:,state] = 0
            A[dim][obs[dim],state] = 1
        self.A = A
        return A
    
    def update_A(self, obs, qs=None):
        """
        Update approximate posterior beliefs about Dirichlet parameters that parameterise the observation likelihood or ``A`` array.

        Parameters
        ----------
        observation: ``list`` or ``tuple`` of ints
            The observation input. Each entry ``observation[m]`` stores the index of the discrete
            observation for modality ``m``.

        Returns
        -----------
        qA: ``numpy.ndarray`` of dtype object
            Posterior Dirichlet parameters over observation self (same shape as ``A``), after having updated it with observations.
        """
        if qs is None:
            qs = self.qs
        qA = update_obs_likelihood_dirichlet(
            self.pA, 
            self.A, 
            obs, 
            qs, 
            self.lr_pA, 
            self.modalities_to_learn
        )

        self.pA = qA # set new prior to posterior
        self.A = utils.norm_dist_obj_arr(qA) # take expected value of posterior Dirichlet parameters to calculate posterior over A array

        return qA
    
    def update_B(self,qs:np.ndarray, qs_prev:np.ndarray, action:int, lr_pB:int=None)-> np.ndarray:
        """
        Updates the posterior Dirichlet parameters (`pB`) that parameterize the transition likelihood (`B`).

        This function refines the transition model by incorporating new posterior beliefs about states (`qs`),
        previous state beliefs (`qs_prev`), and the most recent action taken. The update is performed using 
        a Dirichlet-multinomial approach, ensuring a smooth adaptation of transition probabilities.

        Parameters
        ----------
        qs : numpy.ndarray
            Marginal posterior beliefs over hidden states at the current time step.

        qs_prev : numpy.ndarray
            Marginal posterior beliefs over hidden states at the previous time step.

        action : int
            The most recent action taken by the agent, which affects transition updates.

        lr_pB : int, optional
            Learning rate for updating `pB`. If not specified, defaults to `self.lr_pB`.

        Returns
        -------
        qB : numpy.ndarray
            Updated posterior Dirichlet parameters over transition probabilities (`B`). 
            This has the same shape as `B` but now incorporates learned state-action transitions.

        Notes
        -----
        - The update is computed using the `update_state_likelihood_dirichlet` function, 
        which adjusts `pB` based on the observed transitions.
        - The function ensures that `qB` does not contain negative values by applying a failsafe correction.
        - Transition probabilities (`B`) are normalized after updating `pB` to maintain a valid probability distribution.
        - If `lr_pB` is negative, a failsafe mechanism prevents `qB` from dropping below a minimum threshold (0.005).
        - The updated `qB` is stored as `self.pB`, and `B` is re-normalized for future inference.

        """
        
        if lr_pB is None:
            lr_pB = self.lr_pB
        
        qB = update_state_likelihood_dirichlet(
            self.pB,
            self.B,
            [action],
            qs,
            qs_prev,
            lr_pB,
            self.factors_to_learn
        )
    
        qB[0] = np.maximum(qB[0], 0.005) #No negative value (failsafe because of lr possibly negative)
        #no 0 values because 0 values can't variate anymore
        self.pB = qB # set new prior to posterior
        self.B = utils.norm_dist_obj_arr(qB)  # take expected value of posterior Dirichlet parameters to calculate posterior over B array
        return qB

    def update_A_dim_given_obs(self, obs:list,null_proba:list=[True]) -> np.ndarray:
        ''' 
        Verify if the observations are new and fit into the current A shape.
        If not, increase A shape in observation (n row) only.
        '''
        A = self.A
        num_obs, num_states, num_modalities, num_factors = utils.get_model_dimensions(A=A)
        # Calculate the maximum dimension increase needed across all modalities
        dim_add = [int(max(0,obs[m] + 1 - num_obs[m])) for m in range(num_modalities)]
        # Update matrices size
        for m in range(num_modalities):
            A[m] = update_A_matrix_size(A[m], add_ob=dim_add[m], null_proba=null_proba[m])
            if self.pA is not None:
                self.pA[m] = utils.dirichlet_like(utils.to_obj_array(A[m]), scale=1)[0]
        num_obs, num_states, num_modalities, num_factors = utils.get_model_dimensions(A=A)
        self.num_obs = num_obs
        self.A = A
        return A
    
    def update_A_dim_given_pose(self, pose_idx:int,null_proba:bool=True) -> np.ndarray:
        ''' 
        Verify if the observations are new and fit into the current A shape.
        If not, increase A shape and associate those observations with the newest state generated.
        If yes, search for the first empty column available and fill it with new inferred position (pose_idx)
        '''
        A = self.A
        num_obs, num_states, num_modalities, num_factors = utils.get_model_dimensions(A=A)
        if pose_idx >= max(num_states):
            # Calculate the maximum dimension increase needed across all modalities
            dim_add = int(max(0,pose_idx + 1 - num_obs[num_modalities-1]))
            # Update matrices size
            #and associate new observations with the newest state generated
            if dim_add > 0:
                A[0] = update_A_matrix_size(A[0], add_state=dim_add, null_proba=null_proba)
                if num_modalities > 1:
                    A[1] = update_A_matrix_size(A[1], add_ob=dim_add, add_state=dim_add, null_proba=null_proba)
                    self.num_obs[1] = A[1].shape[0]
        if num_modalities > 1:
            #columns_wthout_data = np.sort(np.append(np.where(np.all(A[1] == 1/A[1].shape[0], axis=0))[0], np.where(np.all(A[1] == 0, axis=0))[0]))
            A[1][:, pose_idx] = 0
            A[1][pose_idx, pose_idx] = 1
            

        if self.pA is not None:
            self.pA = utils.dirichlet_like(utils.to_obj_array(A), scale=1)
                        
        self.num_states = [A[0].shape[1]]
        self.A = A
        return A
    
    def update_B_dim_given_A(self)-> np.ndarray:
        """ knowing A dimension, update B state dimension to match"""
        B = self.B
        add_dim = self.A[0].shape[1]-B[0].shape[1]
        if add_dim > 0: 
            #increase B dim
            B = update_B_matrix_size(B, add= add_dim)
            self.pB = update_B_matrix_size(self.pB, add= add_dim, alter_weights=True)
            if len(self.qs) > 1:
                for seq in self.qs:
                    for subseq in seq:
                        subseq[0] = np.append(subseq[0], [0] * add_dim)
            else:
            
                self.qs[0] = np.append(self.qs[0],[0]*add_dim)
        
        self.num_states = [B[0].shape[0]]
        self.B = B
        return B
    
    def update_believes_with_obs(self, Qs:list, action:int, obs:list)-> None:
        """
        Updates the model's transition (`B`) and observation (`A`) matrices using the given 
        posterior beliefs over states (`Qs`), action taken, and new observation.

        Parameters
        ----------
        Qs : list
            The updated posterior beliefs over hidden states.

        action : int
            The action taken at the current step.

        obs : list
            The observed sensory input at the current step.

        Returns
        -------
        None
            Updates the transition (`B`) and observation (`A`) matrices in-place.

        Notes
        -----
        - If `qs_hist` is available, it retrieves the previous belief state and ensures consistency 
        in dimensionality before updating `B` using `Qs`.
        - If the transition resulted in a change of state, the function also updates `B` for 
        the reverse transition using the inverse action.
        - After updating `B`, the function re-infers the new state (`Qs`) based on `obs` and 
        updates `A` accordingly.
        """
        if len(self.qs_hist) > 0:#secutity check
            qs_hist = self.qs_hist[-1]
            qs_hist[0] = np.append(qs_hist[0],[0]*\
                                   (len(Qs[0])-len(qs_hist[0])))
            self.update_B(Qs, qs_hist, action, lr_pB = 10) 
            #2 WAYS TRANSITION UPDATE (only if T to diff state)
            if np.argmax(qs_hist[0]) != np.argmax(Qs[0]):
                a_inv = reverse_action(self.possible_actions, action)
                self.update_B(qs_hist, Qs, a_inv, lr_pB = 7)
        Qs = self.infer_states(obs) 
        self.update_A(obs, Qs)

    def update_qs_dim(self, qs:np.array=None, update_qs_memory:bool=True)->np.ndarray:
        if qs is None:
            qs = self.qs[:]
        if len(qs[0]) < self.B[0].shape[0]:
            qs[0] = np.append(qs[0],[0]*(self.B[0].shape[0]-len(qs[0])))

        if update_qs_memory:       
            self.qs = qs
            for p_step, past_qs in enumerate(self.qs_hist):
                if len(past_qs[0]) < self.B[0].shape[0]:
                    past_qs[0] = np.append(past_qs[0],[0]*(self.B[0].shape[0]-len(past_qs[0])))
                    self.qs_hist[p_step]= past_qs
        return qs
    
    def update_C_dim(self):
        if self.C is not None:
            num_obs, num_states, num_modalities, num_factors = utils.get_model_dimensions(A=self.A) 
            for m in range(num_modalities):
                if self.C[m].shape[0] < num_obs[m]:
                    self.C[m] = np.append(self.C[m], [0]*(num_obs[m]- self.C[m].shape[0]))
    
    def update_preference(self, obs:list=[-1,-1], pref_weight:float=1.0):
        """given a list of observations (must fill all expected observation. If no preference enters -1) we fill C with thos as preference. 
        If we have a partial preference over several observations, 
        then the given observation should be an integer < 0, the preference will be a null array 
        """
        if isinstance(obs, list):
            self.update_A_dim_given_obs(obs, null_proba=[False]*len(obs))

            C = self._construct_C_prior()

            for modality, ob in enumerate(obs):
                if ob >= 0:
                    self.preferred_ob[modality] = ob
                    ob_processed = utils.process_observation(ob, 1, [self.num_obs[modality]])
                    ob = utils.to_obj_array(ob_processed)
                else:
                    ob = utils.obj_array_zeros([self.num_obs[modality]])
                C[modality] = np.array(ob[0]) * pref_weight

            if not isinstance(C, np.ndarray):
                raise TypeError(
                    'C vector must be a numpy array'
                )
            self.C = utils.to_obj_array(C)

            assert len(self.C) == self.num_modalities, f"Check C vector: number of sub-arrays must be equal to number of observation modalities: {self.num_modalities}"

            for modality, c_m in enumerate(self.C):
                assert c_m.shape[0] == self.num_obs[modality], f"Check C vector: number of rows of C vector for modality {modality} should be equal to {self.num_obs[modality]}"
        else:
            self.preferred_ob = [-1,-1]
            self.C = self._construct_C_prior()

    #====== UPDATE MODEL ======#
    def update_transitions_both_ways(self,qs:np.ndarray, next_qs:np.ndarray, action_id:int, \
                                     direct_lr_pB:int, reverse_lr_pB:int)-> None:
        
        a_inv = reverse_action(self.possible_actions, action_id)
        self.update_B(next_qs, qs, action_id, lr_pB = direct_lr_pB) 
        self.update_B(qs, next_qs, a_inv, lr_pB = reverse_lr_pB)
   
    def update_imagined_translation(self, qs:np.ndarray, action_jump:int, n_actions:int, action_id:int, cur_pose:list, \
                                    min_dist_to_next_node:int, obstacle_dist_per_actions:int):
        """
        Updates the model's transition probabilities (`B` matrix) from current pose to imagined poses and between imagined poses up to 'action_jump' actions away
        It reinforces direct and indirect motion transitions while considering obstacles. 

        Parameters
        ----------
        qs : np.ndarray
            The posterior belief over states before taking the action.

        action_jump : int
            The range of adjacent actions to consider when updating transitions.

        n_actions : int
            The total number of possible actions.

        action_id : int
            The current action being taken.

        cur_pose : list
            The current position in the environment.

        min_dist_to_next_node : int
            The minimum distance required to move to the next node.

        obstacle_dist_per_actions : int
            The distance of obstacles from the current state for each possible action.

        hypo_qs : list or None
            The hypothetical belief state over hidden states, used when reinforcing indirect motion. 
            If `None`, direct motion updates are applied.

        Returns
        -------
        None
            Updates the transition probabilities (`B` matrix) in-place.

        Notes
        -----

        """
        #1) We get current physical pose info + the next imagined pose info given action_id
        _, next_pose_id = self.determine_next_pose(cur_pose, action_id, min_dist_to_next_node) #from odom to next imagined pose
        next_pose = self.PoseMemory.id_to_pose(next_pose_id) #get memorised pose (not the approximated one)
        cur_pose_id = self.PoseMemory.pose_to_id(cur_pose) #get odom pose id
        next_state_ob_dist = obstacle_dist_per_actions[action_id] #to check if ob between physical pose to next imagined pose
        
        prev_action = -1
        print('__')
        for offset in range(action_jump, -action_jump - 1, -1):
            action_adjacent = action_id + offset
            print('action_adjacent, offset', action_adjacent, offset)
            #restraingning action between possible actions numbers
            if action_adjacent < 0 :
                action_adjacent = n_actions +action_adjacent
            else:
                action_adjacent %= (n_actions)
            #from physical pose, get next adjacent pose given offset action 
            next_adjacent_pose, next_adjacent_pose_id = self.determine_next_pose(cur_pose, action_adjacent, min_dist_to_next_node)
            #if no adjacent pose, nothing to do
            print('next_adjacent_pose_id', next_adjacent_pose_id,'next_pose_id', next_pose_id, 'cur_pose_id',cur_pose_id)
            if next_adjacent_pose_id < 0 or next_adjacent_pose_id==cur_pose_id:
                continue
            #if adjacent pose exists, then we get it's state (obtained from Transitioning from physical state to this adjacent state)
            adjacent_qs = self.infer_states([next_adjacent_pose_id], np.array([action_adjacent]), save_hist=False, partial_ob=1, qs=qs)
            adjacent_state_dist_to_ob = obstacle_dist_per_actions[action_adjacent] #get if ob at this adjacent pose
            print('offset', offset, 'action_adjacent', action_adjacent, 'adjacent_state_dist_to_ob', round(adjacent_state_dist_to_ob,2))
            #We correct to pose ID pose, to be sure it matches
            next_adjacent_pose = self.PoseMemory.id_to_pose(next_adjacent_pose_id) #get memorised pose (not the approximated one)
            
            #If known state and this is the direct transition from physical to an already existing state 
            if offset == 0 : #if direct motion from current state to another state  
                reference_qs = qs
                action = action_adjacent
                direct_lr_pB = 5
                reverse_lr_pB = 3
                pose_in_action_range = self.PoseMemory.pose_in_action_range(action, next_pose, odom= cur_pose)
                print('direct transition from new current odom', cur_pose_id, 'to', next_pose_id)
                
            #If this transition is a lateral transition 
            elif offset != 0 and next_adjacent_pose_id != next_pose_id: #if indirect motion, we don't want to reinforce 'stay' motion with wrong action
                reference_qs = self.infer_states([next_pose_id], np.array([action_id]), save_hist=False, partial_ob=1, qs=qs) #next direct imagined pose qs
                angle = angle_turn_from_pose_to_p(pose = next_pose, goal_pose= next_adjacent_pose, in_deg=True)
                action = self.determine_action_given_angle_deg(angle)
                direct_lr_pB = 1
                reverse_lr_pB = 1
                pose_in_action_range = self.PoseMemory.pose_in_action_range(action, next_adjacent_pose, odom= next_pose)
                #Just to avoid reinforcing same link several times (can happens if we check pose to id only considering distance)
                if prev_action == action:
                    print('already updated that transition')
                    continue
                prev_action = action
                print('lateral transition from imagined pose',next_pose_id, 'to', next_adjacent_pose_id)
            else:
                continue
            print('pose_in_action_range', pose_in_action_range, 'action', action,'next_pose', next_adjacent_pose, 'odom', next_pose)
            #If the pose is not in this action range, we don't enforce it + we can't have the poses being unreachable.
            if pose_in_action_range and adjacent_state_dist_to_ob > min_dist_to_next_node and next_state_ob_dist > min_dist_to_next_node :
                # Positive LR
                self.update_transitions_both_ways(reference_qs, adjacent_qs, action, direct_lr_pB=direct_lr_pB, reverse_lr_pB=reverse_lr_pB)
            elif pose_in_action_range:
                print('negative reinforcement')
                # Negative LR
                self.update_transitions_both_ways(reference_qs, adjacent_qs, action, direct_lr_pB=-direct_lr_pB, reverse_lr_pB=-reverse_lr_pB)

    def update_transition_nodes(self, obstacle_dist_per_actions:list)-> None:
        ''' 
        For each new pose observation, add a ghost state and update the estimated transition and observation for that ghost state.
        '''
        print('Ghost nodes process:')
        action_jump = int((len(self.possible_actions)-1) / 6)
        sure_about_data_until_this_state = 1
        ''' 
            TODO LIST: 
            1) Check if transition possible 
            IF impossible motion:
                2) increase transition from current state to current state for this action (both ways)
                3) if obstacle, check if there is a transition existing for this action and decrease state transition (both ways)
            go to next action
            Else: 
                4) infer new pose in that direction 
                5) check if a pose exist in that direction (margin of zone of influence)
                6) if yes, increase transition prob to existing pose with that action
                7) if no, 
                    7') increase all matrices IF NEED BE
                    7'')create new node 
                8) check if previous and next action have obstacle. If no, link previous/next pose node to current pose node
                9) from this node, check if action still possible further with increased zone of influence + margin (thus until we reach an obstacle) 
            skip next action (as we want 6 nodes around if no obstacle anywhere)
            '''
        
        if "STAY" in self.possible_actions.values():
            n_actions = len(self.possible_actions) -1
        else:
            n_actions = len(self.possible_actions) 
        qs = self.get_belief_over_states()
        min_dist_to_next_node = self.influence_radius + self.robot_dim/2#/3 to consider -a little- robot_dim when adding nodes.as_integer_ratio
        
        for action_id in range(n_actions):
            print('______________________________')
            hypo_qs = None
            state_step = 1
            prev_step_qs = qs[:]
            no_obstacle = True
            cur_pose = self.PoseMemory.get_odom().copy()
            cur_ref_pose = cur_pose.copy()
            pose_in_action_range = True
            
            #9)
            #The second element is just to avoid any risk of infinity loop
            while no_obstacle and state_step<=self.lookahead_node_creation:
                next_state_min_dist_to_next_node = self.influence_radius *state_step + self.robot_dim/2
                #1)
                #Is obstacle too close?
                print('for action', action_id, 'obstacle', obstacle_dist_per_actions[action_id], 'min_dist for new state', next_state_min_dist_to_next_node)
                if obstacle_dist_per_actions[action_id] <=  next_state_min_dist_to_next_node :
                    no_obstacle = False 
                    #Only enforce the loop back to current pose if it's a direct motion
                    if state_step <= sure_about_data_until_this_state:
                        #2)
                        print('enforcing motion:',action_id,' leads to current state')
                        self.update_B(qs, qs, action_id, lr_pB = 10)   
                else:
                    #4)
                    next_pose, next_pose_id = self.determine_next_pose(cur_ref_pose, action_id, min_dist_to_next_node)
                    print('next_pose', next_pose, self.PoseMemory.get_poses_from_memory().copy())
                    #5) ->7)  
                    if next_pose_id < 0 and pose_in_action_range:
                        next_pose_id = self.PoseMemory.pose_to_id(next_pose) 
                        print('creating new node in position', next_pose_id)
                        #7')
                        self.update_A_dim_given_pose(next_pose_id,null_proba=True)
                        self.update_B_dim_given_A()
                        self.update_C_dim()
                        prev_step_qs = self.update_qs_dim(prev_step_qs,update_qs_memory=False)
                        #7'')
                        hypo_qs = self.infer_states([next_pose_id], np.array([action_id]), partial_ob=1, save_hist=False, qs=prev_step_qs)
                        self.update_agent_state_mapping(tuple(next_pose[:2]), [-1, next_pose_id], hypo_qs[0])
                        #We don't want lateral state transition updated when we are extrapolating further than "sure_about_data_until_this_state"
                        #plus we only want the action continuing in a straight line from physical current pose. 
                        #becquse the beam rqnge grows bigger as the vectors are longer.                            
                    else:
                        print('pose existing nearby as', next_pose_id,'not creating new node')
                        hypo_qs = self.infer_states([next_pose_id], np.array([action_id]), partial_ob=1, save_hist=False, qs=prev_step_qs)

                    if state_step > sure_about_data_until_this_state and pose_in_action_range:
                            print('DIRECT MOTION AT STATE STEP',state_step)
                            self.update_imagined_translation(prev_step_qs[:], 0, n_actions, action_id, cur_ref_pose, \
                                        min_dist_to_next_node, obstacle_dist_per_actions)
                    prev_step_qs = hypo_qs[:]
                    cur_ref_pose = self.PoseMemory.id_to_pose(next_pose_id)
                    pose_in_action_range = self.PoseMemory.pose_in_action_range(action_id, cur_ref_pose, odom= cur_pose, influence_radius=next_state_min_dist_to_next_node)#doesn't work
                    print('cur_ref_pose', cur_ref_pose, 'can be reached from ', cur_pose, 'with action', action_id,'?:', pose_in_action_range)
                    
                state_step +=1
            #3), 5)->6)with offset 0 and 8)
            qs = self.update_qs_dim(qs)
            self.update_imagined_translation(qs[:], action_jump, n_actions, action_id, cur_pose, \
                                   min_dist_to_next_node, obstacle_dist_per_actions)
               
    def agent_step_update(self, action_id:int, obs:list, logs=None)->None:
        """
        Updates the agent's belief state, transition probabilities, and learned environment 
        model based on the given action and observations.

        This method performs the following steps:
        1. **Infer new pose**: Updates the agent's estimated position.
        2. **Update observation model (A) and control model (C)**: Adjusts probability distributions 
        to incorporate new observations.
        3. **Update transition model (B)**: Modifies transition probabilities based on inferred states.
        4. **Update belief states**: Updates internal beliefs given the latest observations.
        5. **Update state mapping for visualization**: Stores the agent's inferred position and 
        corresponding belief state.
        6. **Update transition nodes**: Modifies state transitions based on perceived obstacles.
        7. **Ensure stationary transitions**: If a 'STAY' action exists, enforces it in the transition model.

        Parameters:
            action_id (int): The index of the action taken by the agent.
            obs (list): The list of observations, expected to contain:
                - primary_ob (int): The primary observed feature (e.g., color).
                - pose_id (int, optional): The ID of the inferred pose. 
                - obstacles_dist_per_action_range (list): Distances to obstacles for each action.

        Returns:
            None
        """
        #we could get action_id from ours.action instead?

        #We only update A and B if we have inferred a current pose
        #Thus until doubt over current loc is not solved, we don't update internal self
        if self.current_pose != None:
            #Just for memory sake
            primary_ob = obs[0]
            if isinstance(obs[1],int):
                pose_id = obs[1]
            else:
                pose_id = self.PoseMemory.pose_to_id(self.current_pose)

            obstacles_dist_per_action_range = obs[-1]
            
            #1. INFER NEW POSE (should be after motion and before update)
            # pose = self.PoseMemory.id_to_pose(pose_id)
            # self.current_pose = self.infer_pose(pose) #Not sure it shouold be here. in case i want to give whatever...
            
            observations = [primary_ob,pose_id]
            if logs:
                logs.info('observations pose %f, action %f, ob_id %f, obstacles %s' % (pose_id, action_id, primary_ob, str(obstacles_dist_per_action_range)))
            #2. UPDATE A C DIM IF NEW OB
            self.update_A_dim_given_obs(observations, null_proba=[True,False])
            self.update_C_dim()
            #updating B in case pose_id new
            self.update_B_dim_given_A()
            #3. UPDATE BELIEVES GIVEN OBS
            prior = self.infer_states(observations, save_hist=False)

            print('prior on believed state; action', self.action, action_id, \
                'colour_ob:', primary_ob , 'inf pose:',self.current_pose,'prior belief:', prior[0].round(3))
                
            self.update_believes_with_obs(prior,action=action_id, obs=observations)

            posterior = self.infer_states(observations, save_hist=True)
            ## agent_state_mapping for TEST PURPOSES and visualisation
            self.update_agent_state_mapping(tuple(self.current_pose[:2]), observations, posterior[0])
            #4. update all nodes
            self.update_transition_nodes(obstacle_dist_per_actions=obstacles_dist_per_action_range)
            #This is not mandatory, just a gain of time
            if 'STAY' in self.possible_actions.values():
                stay_action = [key for key, value in self.possible_actions.items() if value == 'STAY'][0]
                self.B[0] = set_stationary(self.B[0], stay_action)

In [8]:
ours = Ours_V5_RW(num_obs=2, num_states=2, n_actions=13, influence_radius=0.5,robot_dim=0.3, lookahead_node_creation=2)

In [9]:
stop

NameError: name 'stop' is not defined

# Update model

### TEst init MCTS

In [10]:
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
obstacles = [1.4612406492233276, 2.0710742473602295, 2.7915122509002686, 2.571498155593872, 2.9910919666290283, 2.616743803024292, 2.5963499546051025, 1.2535045146942139, 0.49500948190689087, 0.5026026964187622, 1.1839525699615479, 1.4570804834365845]

ob_id = 0

ours = Ours_V5_RW(num_obs=2, num_states=2, dim=2, \
                    observations=[ob_id], lookahead_policy=5,\
                    n_actions=13, influence_radius=1,\
                    robot_dim=0.3, lookahead_node_creation= 2)


In [ ]:
ours.update_transition_nodes(obstacle_dist_per_actions=obstacles)

In [ ]:
ours.update_preference([0,-1], pref_weight=1)
ours.C

In [23]:
ours.use_states_info_gain = True 
ours.use_utility = False
ours.use_param_info_gain = False #if true, do not use with the other terms

In [25]:
action, data = ours.define_actions_from_MCTS_run(num_step=1, observations=[0])

2025-04-17 12:13:10,331 - INFO - MCTS_Model_Interface initialized with model type: <class '__main__.Ours_V5_RW'>
2025-04-17 12:13:10,333 - INFO - MCTS initialized with exploration parameter c=5, num_simus=25, max_depth=7, policy_alpha=16.0,  action_selection=stochastic
2025-04-17 12:13:10,335 - INFO - Starting MCTS planning from root node 0 for 25 simulations.
2025-04-17 12:13:10,337 - INFO - --- Simulation 1/25 ---
2025-04-17 12:13:10,338 - INFO - --- Selection Phase End (Selected Node: 0) ---
2025-04-17 12:13:10,376 - INFO - Root node children stats: [('child id', 1, 'N', 0, 'T', 0, 'efe_av', 0.0), ('child id', 3, 'N', 0, 'T', 0, 'efe_av', 0.0), ('child id', 5, 'N', 0, 'T', 0, 'efe_av', 0.0), ('child id', 7, 'N', 0, 'T', 0, 'efe_av', 0.0), ('child id', 9, 'N', 0, 'T', 0, 'efe_av', 0.0), ('child id', 11, 'N', 0, 'T', 0, 'efe_av', 0.0), ('child id', 0, 'N', 0, 'T', 0, 'efe_av', 0.0)]
2025-04-17 12:13:10,377 - INFO - --- Simulation 2/25 ---
2025-04-17 12:13:10,378 - INFO - --- Selection

2025-04-17 12:13:10,540 - INFO - Root node children stats: [('child id', 1, 'N', 1, 'T', 0.89, 'efe_av', 0.89), ('child id', 3, 'N', 1, 'T', 1.38, 'efe_av', 1.38), ('child id', 5, 'N', 1, 'T', 0.97, 'efe_av', 0.97), ('child id', 7, 'N', 0, 'T', 0, 'efe_av', 0.0), ('child id', 9, 'N', 0, 'T', 0, 'efe_av', 0.0), ('child id', 11, 'N', 0, 'T', 0, 'efe_av', 0.0), ('child id', 0, 'N', 0, 'T', 0, 'efe_av', 0.0)]
2025-04-17 12:13:10,543 - INFO - --- Simulation 5/25 ---
2025-04-17 12:13:10,545 - INFO - --- Selection Phase End (Selected Node: 7) ---
2025-04-17 12:13:10,606 - INFO - Root node children stats: [('child id', 1, 'N', 1, 'T', 0.89, 'efe_av', 0.89), ('child id', 3, 'N', 1, 'T', 1.38, 'efe_av', 1.38), ('child id', 5, 'N', 1, 'T', 0.97, 'efe_av', 0.97), ('child id', 7, 'N', 1, 'T', 0.79, 'efe_av', 0.79), ('child id', 9, 'N', 0, 'T', 0, 'efe_av', 0.0), ('child id', 11, 'N', 0, 'T', 0, 'efe_av', 0.0), ('child id', 0, 'N', 0, 'T', 0, 'efe_av', 0.0)]
2025-04-17 12:13:10,609 - INFO - --- Simu

2025-04-17 12:13:10,810 - INFO - Root node children stats: [('child id', 1, 'N', 1, 'T', 0.89, 'efe_av', 0.89), ('child id', 3, 'N', 1, 'T', 1.38, 'efe_av', 1.38), ('child id', 5, 'N', 1, 'T', 0.97, 'efe_av', 0.97), ('child id', 7, 'N', 1, 'T', 0.79, 'efe_av', 0.79), ('child id', 9, 'N', 1, 'T', 1.09, 'efe_av', 1.09), ('child id', 11, 'N', 1, 'T', 0.62, 'efe_av', 0.62), ('child id', 0, 'N', 1, 'T', 0.29, 'efe_av', 0.29)]
2025-04-17 12:13:10,812 - INFO - --- Simulation 9/25 ---
2025-04-17 12:13:10,813 - INFO - --- Selection Phase End (Selected Node: 4) ---
2025-04-17 12:13:10,850 - INFO - Root node children stats: [('child id', 1, 'N', 1, 'T', 0.89, 'efe_av', 0.89), ('child id', 3, 'N', 2, 'T', 2.22, 'efe_av', 1.11), ('child id', 5, 'N', 1, 'T', 0.97, 'efe_av', 0.97), ('child id', 7, 'N', 1, 'T', 0.79, 'efe_av', 0.79), ('child id', 9, 'N', 1, 'T', 1.09, 'efe_av', 1.09), ('child id', 11, 'N', 1, 'T', 0.62, 'efe_av', 0.62), ('child id', 0, 'N', 1, 'T', 0.29, 'efe_av', 0.29)]
2025-04-17 12

2025-04-17 12:13:11,043 - INFO - --- Simulation 14/25 ---
2025-04-17 12:13:11,045 - INFO - --- Selection Phase End (Selected Node: 1) ---
2025-04-17 12:13:11,097 - INFO - Root node children stats: [('child id', 1, 'N', 2, 'T', 1.63, 'efe_av', 0.81), ('child id', 3, 'N', 2, 'T', 2.22, 'efe_av', 1.11), ('child id', 5, 'N', 2, 'T', 2.58, 'efe_av', 1.29), ('child id', 7, 'N', 2, 'T', 3.85, 'efe_av', 1.93), ('child id', 9, 'N', 2, 'T', 3.08, 'efe_av', 1.54), ('child id', 11, 'N', 2, 'T', 2.01, 'efe_av', 1.01), ('child id', 0, 'N', 1, 'T', 0.29, 'efe_av', 0.29)]
2025-04-17 12:13:11,098 - INFO - --- Simulation 15/25 ---
2025-04-17 12:13:11,099 - INFO - --- Selection Phase End (Selected Node: 1) ---
2025-04-17 12:13:11,142 - INFO - Root node children stats: [('child id', 1, 'N', 2, 'T', 1.63, 'efe_av', 0.81), ('child id', 3, 'N', 2, 'T', 2.22, 'efe_av', 1.11), ('child id', 5, 'N', 2, 'T', 2.58, 'efe_av', 1.29), ('child id', 7, 'N', 2, 'T', 3.85, 'efe_av', 1.93), ('child id', 9, 'N', 2, 'T', 3.

2025-04-17 12:13:11,278 - INFO - Root node children stats: [('child id', 1, 'N', 2, 'T', 1.63, 'efe_av', 0.81), ('child id', 3, 'N', 2, 'T', 2.22, 'efe_av', 1.11), ('child id', 5, 'N', 3, 'T', 3.68, 'efe_av', 1.23), ('child id', 7, 'N', 3, 'T', 4.76, 'efe_av', 1.59), ('child id', 9, 'N', 3, 'T', 5.26, 'efe_av', 1.75), ('child id', 11, 'N', 2, 'T', 2.01, 'efe_av', 1.01), ('child id', 0, 'N', 2, 'T', 0.93, 'efe_av', 0.46)]
2025-04-17 12:13:11,279 - INFO - --- Simulation 19/25 ---
2025-04-17 12:13:11,282 - INFO - --- Selection Phase End (Selected Node: 5) ---
2025-04-17 12:13:11,333 - INFO - Root node children stats: [('child id', 1, 'N', 2, 'T', 1.63, 'efe_av', 0.81), ('child id', 3, 'N', 3, 'T', 4.03, 'efe_av', 1.34), ('child id', 5, 'N', 3, 'T', 3.68, 'efe_av', 1.23), ('child id', 7, 'N', 3, 'T', 4.76, 'efe_av', 1.59), ('child id', 9, 'N', 3, 'T', 5.26, 'efe_av', 1.75), ('child id', 11, 'N', 2, 'T', 2.01, 'efe_av', 1.01), ('child id', 0, 'N', 2, 'T', 0.93, 'efe_av', 0.46)]
2025-04-17 1

2025-04-17 12:13:11,491 - INFO - --- Simulation 23/25 ---
2025-04-17 12:13:11,492 - INFO - --- Selection Phase End (Selected Node: 3) ---
2025-04-17 12:13:11,543 - INFO - Root node children stats: [('child id', 1, 'N', 3, 'T', 3.22, 'efe_av', 1.07), ('child id', 3, 'N', 3, 'T', 4.03, 'efe_av', 1.34), ('child id', 5, 'N', 3, 'T', 3.68, 'efe_av', 1.23), ('child id', 7, 'N', 3, 'T', 4.76, 'efe_av', 1.59), ('child id', 9, 'N', 4, 'T', 6.88, 'efe_av', 1.72), ('child id', 11, 'N', 3, 'T', 4.71, 'efe_av', 1.57), ('child id', 0, 'N', 3, 'T', 1.67, 'efe_av', 0.56)]
2025-04-17 12:13:11,545 - INFO - --- Simulation 24/25 ---
2025-04-17 12:13:11,546 - INFO - --- Selection Phase End (Selected Node: 8) ---
2025-04-17 12:13:11,586 - INFO - Root node children stats: [('child id', 1, 'N', 3, 'T', 3.22, 'efe_av', 1.07), ('child id', 3, 'N', 3, 'T', 4.03, 'efe_av', 1.34), ('child id', 5, 'N', 3, 'T', 3.68, 'efe_av', 1.23), ('child id', 7, 'N', 4, 'T', 5.51, 'efe_av', 1.38), ('child id', 9, 'N', 4, 'T', 6.

In [ ]:
ours.action = np.array([12])

In [ ]:
ours.get_belief_over_states()[0].round(3)

In [ ]:
prior = ours.infer_states([0,0], save_hist=False)
prior

In [ ]:
data

## TEST 0 WITH OBSTACLE - pose_obs_test0

### Init

In [ ]:
import pandas as pd
csvfile = pd.read_csv('/home/idlab332/workspace/ros_ws/pose_obs_test0_v3.csv')
csvfile

#START POSE
ob_id = 0 #csvfile['ob_id'].values[-1]
odom_theta = float(csvfile['theta'].values[2])
odom = [0.0,0.0, odom_theta]

p_idx = ours.PoseMemory.pose_to_id(odom)
obstacle_dists = eval(csvfile['ob_dists'].values[2])

In [ ]:
qs = ours.get_belief_over_states() #self.qs
# agent_state_mapping for TEST PURPOSES
ours.update_agent_state_mapping(tuple(odom[:2]), [ob_id,p_idx], qs[0])
ours.update_transition_nodes(obstacle_dist_per_actions=obstacle_dists)

### Step 1

In [ ]:
#STEP1 
action_step1 = 0
ob_id_step1 = 1 #csvfile['ob_id'].values[-1]
excel_step = 5
pose_x = float(csvfile['Pose x for agent'].values[excel_step])
pose_y = float(csvfile['Pose Y for agent'].values[excel_step])
theta = float(csvfile['theta'].values[excel_step])
p_idx_step1 = ours.PoseMemory.pose_to_id([pose_x,pose_y], save_in_memory=False)
pose = ours.PoseMemory.id_to_pose(p_idx_step1)
odom_step1 = [pose[0],pose[1], theta]
obstacle_dists_step1 = eval(csvfile['ob_dists'].values[excel_step])

In [ ]:
ours.agent_manual_action_step(action_step1)
ours.agent_step_update(action_step1, [ob_id_step1,p_idx_step1,obstacle_dists_step1])

In [ ]:
# stop

### Step 2 

In [ ]:
#STEP2
action_step2 = 2
ob_id_step2 = 2 #csvfile['ob_id'].values[-1]
excel_step = 6
pose_x = float(csvfile['Pose x for agent'].values[excel_step])
pose_y = float(csvfile['Pose Y for agent'].values[excel_step])
theta = float(csvfile['theta'].values[excel_step])
p_idx_step2 = ours.PoseMemory.pose_to_id([pose_x,pose_y], save_in_memory=False)
pose = ours.PoseMemory.id_to_pose(p_idx_step2)
odom_step2 = [pose[0],pose[1], theta]
obstacle_dists_step2 = eval(csvfile['ob_dists'].values[excel_step])
ob_id_step2 ,p_idx_step2, 

In [ ]:
ours.agent_manual_action_step(action_step2)
ours.agent_step_update(action_step2, [ob_id_step2,p_idx_step2,obstacle_dists_step2])


In [ ]:
# stop

### step 3

In [ ]:
#STEP3 NO MOTION
action_step3 = 12
ob_id_step3 = 2 #csvfile['ob_id'].values[-1]

excel_step = 6
pose_x = float(csvfile['Pose x for agent'].values[excel_step])
pose_y = float(csvfile['Pose Y for agent'].values[excel_step])
theta = float(csvfile['theta'].values[excel_step])
p_idx_step3 = ours.PoseMemory.pose_to_id([pose_x,pose_y], save_in_memory=False)
pose = ours.PoseMemory.id_to_pose(p_idx_step3)
odom_step3 = [pose[0],pose[1], theta]
obstacle_dists_step3 = eval(csvfile['ob_dists'].values[excel_step])

In [ ]:
ours.agent_manual_action_step(action_step3)
ours.agent_step_update(action_step3, [ob_id_step3,p_idx_step3,obstacle_dists_step3])


In [ ]:
# stop

### step 4

In [ ]:
#STEP4
action_step4 = 5
ob_id_step4 = 3 #csvfile['ob_id'].values[-1]

#with obstacle
excel_step = 8
#Without obstacle
# excel_step = 9

pose_x = float(csvfile['Pose x for agent'].values[excel_step])
pose_y = float(csvfile['Pose Y for agent'].values[excel_step])
theta = float(csvfile['theta'].values[excel_step])
p_idx_step4 = ours.PoseMemory.pose_to_id([pose_x,pose_y], save_in_memory=False)
pose = ours.PoseMemory.id_to_pose(p_idx_step4)
odom_step4 = [pose[0],pose[1], theta]
obstacle_dists_step4= eval(csvfile['ob_dists'].values[excel_step])

In [ ]:
ours.agent_manual_action_step(action_step4)
ours.agent_step_update(action_step4, [ob_id_step4,p_idx_step4,obstacle_dists_step4])


## TEST 1 SQUARE MOTION NO OB - 

### init

In [11]:
import pandas as pd
csvfile = pd.read_csv('/home/idlab332/workspace/ros_ws/pose_obs_test1_v3.csv')
# csvfile

In [12]:
#START POSE
ob_id = 0 #csvfile['ob_id'].values[-1]
odom_theta = float(csvfile['theta'].values[0])
odom = [0.0,0.0, odom_theta]

p_idx = ours.PoseMemory.pose_to_id(odom)
obstacle_dists = eval(csvfile['ob_dists'].values[0])

In [13]:
qs = ours.get_belief_over_states() #self.qs
# agent_state_mapping for TEST PURPOSES
ours.update_agent_state_mapping(tuple(odom[:2]), [ob_id,p_idx], qs[0])
ours.update_transition_nodes(obstacle_dist_per_actions=obstacle_dists)

Ghost nodes process:
______________________________
for action 0 obstacle 4.487496201197306 min_dist for new state 0.65
action, next pose and id 0 [0.63, 0.17] -1
next_pose [0.63, 0.17] [(0, 0)]
creating new node in position 1
cur_ref_pose (0.63, 0.17) can be reached from  [0, 0, 0.0] with action 0 ?: True
for action 0 obstacle 4.487496201197306 min_dist for new state 1.15
action, next pose and id 0 [1.26, 0.34] -1
next_pose [1.26, 0.34] [(0, 0), (0.63, 0.17)]
creating new node in position 2
DIRECT MOTION AT STATE STEP 2
action, next pose and id 0 [1.26, 0.34] 2
__
action_adjacent, offset 0 0
action, next pose and id 0 [1.26, 0.34] 2
next_adjacent_pose_id 2 next_pose_id 2 cur_pose_id 1
offset 0 action_adjacent 0 adjacent_state_dist_to_ob 4.49
direct transition from new current odom 1 to 2
pose_in_action_range True action 0 next_pose (1.26, 0.34) odom (1.26, 0.34)
update_B: a 0 qs[factor] [0.001 0.001 0.998] qs_prev[factor] [0. 1. 0.]
update_B: a 6 qs[factor] [0. 1. 0.] qs_prev[factor] 

### step 1

In [14]:
#STEP1 
action_step1 = 10
ob_id_step1 = 1 
excel_step = 1
# p_id = int(csvfile['state'].values[excel_step])
pose_x = float(csvfile['Pose x for agent'].values[excel_step])
pose_y = float(csvfile['Pose Y for agent'].values[excel_step])
theta = float(csvfile['theta'].values[excel_step])
p_idx_step1 = ours.PoseMemory.pose_to_id([pose_x,pose_y], save_in_memory=False)
pose = ours.PoseMemory.id_to_pose(p_idx_step1)

odom_step1 = [pose[0],pose[1], theta]

obstacle_dists_step1 = eval(csvfile['ob_dists'].values[excel_step])

In [15]:
pose_x, pose_y, odom_step1, p_idx_step1

(0.46, -0.46, [0.46, -0.46, -0.0430574688417317], 11)

In [16]:
ours.agent_manual_action_step(action_step1)
ours.agent_step_update(action_step1, [ob_id_step1,p_idx_step1,obstacle_dists_step1])

# ours.PoseMemory.update_odom_given_pose(odom_step1[:2])
# ours.current_pose = ours.PoseMemory.get_odom(as_tuple=True)
# ours.update_A_dim_given_obs([ob_id_step1,p_idx_step1], null_proba=[False,False])
# Qs = ours.infer_states([ob_id_step1,p_idx_step1], save_hist=False)
# print('prior on believed state; action', ours.action, 'colour_ob:', ob_id_step1, 'inf pose:',odom_step1,'belief:', Qs[0].round(3))
            
# ours.update_believes_with_obs(Qs,action=action_step1, obs=[ob_id_step1,p_idx_step1])
# qs = ours.get_belief_over_states()

# print('qs',qs[0].round(3))
# # # agent_state_mapping for TEST PURPOSES
# ours.update_agent_state_mapping(tuple(odom_step1[:2]), [ob_id_step1,p_idx_step1], qs[0])
# ours.update_transition_nodes(obstacle_dist_per_actions=obstacle_dists_step1)

prior on believed state; action [10] 10 colour_ob: 1 inf pose: (0, 0) prior belief: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
update_B: a 10 qs[factor] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.] qs_prev[factor] [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
update_B: a 4 qs[factor] [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] qs_prev[factor] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
Ghost nodes process:
______________________________
for action 0 obstacle 3.8368590831756593 min_dist for new state 0.65
action, next pose and id 0 [0.63, 0.17] 1
next_pose [0.63, 0.17] [(0, 0), (0.63, 0.17), (1.26, 0.34), (0.17, 0.63), (0.34, 1.26), (-0.46, 0.46), (-0.92, 0.92), (-0.63, -0.17), (-1.26, -0.34), (-0.17, -0.63), (-0.34, -1.26), (0.46, -0.46), (0.92, -0.92)]
pose existing nearby as 1 not creating new node
cur_ref_pose (0.63, 0.17) can be reached from  [0, 0, 0.0] with action 0 ?: True
for action 0 obstacle 3.8368590831756593 min_dist for new state 1.15
action, next pose and id 0 [1.26, 0.34] 2
next_pose [1.

### step 2

In [17]:
#STEP2
action_step2 = 8
ob_id_step2 = 2 
excel_step = 2

pose_x = float(csvfile['Pose x for agent'].values[excel_step])
pose_y = float(csvfile['Pose Y for agent'].values[excel_step])
theta = float(csvfile['theta'].values[excel_step])
p_idx_step2 = ours.PoseMemory.pose_to_id([pose_x,pose_y], save_in_memory=False)
pose = ours.PoseMemory.id_to_pose(p_idx_step2)
odom_step2 = [pose[0],pose[1], theta]

obstacle_dists_step2 = eval(csvfile['ob_dists'].values[excel_step])
ob_id_step2 ,p_idx_step2, 

(2, -1)

In [18]:
ours.agent_manual_action_step(action_step2)
ours.agent_step_update(action_step2, [ob_id_step2,p_idx_step2,obstacle_dists_step2])

prior on believed state; action [8] 8 colour_ob: 2 inf pose: (0, 0) prior belief: [0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.088 0.001 0.
 0.902]
update_B: a 8 qs[factor] [0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.088 0.001 0.
 0.902] qs_prev[factor] [0.    0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.995
 0.   ]
update_B: a 2 qs[factor] [0.    0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.995
 0.   ] qs_prev[factor] [0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.088 0.001 0.
 0.902]
Ghost nodes process:
______________________________
for action 0 obstacle 4.65392906665802 min_dist for new state 0.65
action, next pose and id 0 [0.63, 0.17] 1
next_pose [0.63, 0.17] [(0, 0), (0.63, 0.17), (1.26, 0.34), (0.17, 0.63), (0.34, 1.26), (-0.46, 0.46), (-0.92, 0.92), (-0.63, -0.17), (-1.26, -0.34), (-0.17, -0.63), (-0.34, -1.26), (0.46, -0.46), (0.92, -0.92)]
pose existing nearby as 1 not creating new node
cur_ref_pose (0.63,

### step 3

In [19]:
#STEP3
action_step3 = 4
ob_id_step3 = 3
excel_step = 3

pose_x = float(csvfile['Pose x for agent'].values[excel_step])
pose_y = float(csvfile['Pose Y for agent'].values[excel_step])
theta = float(csvfile['theta'].values[excel_step])
p_idx_step3 = ours.PoseMemory.pose_to_id([pose_x,pose_y], save_in_memory=False)
pose = ours.PoseMemory.id_to_pose(p_idx_step3)
odom_step3 = [pose[0],pose[1], theta]
obstacle_dists_step3 = eval(csvfile['ob_dists'].values[excel_step])

In [20]:
ours.agent_manual_action_step(action_step3)
ours.agent_step_update(action_step3, [ob_id_step3,p_idx_step3,obstacle_dists_step3])

prior on believed state; action [4] 4 colour_ob: 3 inf pose: (0, 0) prior belief: [0.001 0.001 0.001 0.001 0.001 0.075 0.001 0.001 0.001 0.917 0.    0.
 0.   ]
update_B: a 4 qs[factor] [0.001 0.001 0.001 0.001 0.001 0.075 0.001 0.001 0.001 0.917 0.    0.
 0.   ] qs_prev[factor] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
update_B: a 10 qs[factor] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.] qs_prev[factor] [0.001 0.001 0.001 0.001 0.001 0.075 0.001 0.001 0.001 0.917 0.    0.
 0.   ]
Ghost nodes process:
______________________________
for action 0 obstacle 4.888394737243653 min_dist for new state 0.65
action, next pose and id 0 [0.63, 0.17] 1
next_pose [0.63, 0.17] [(0, 0), (0.63, 0.17), (1.26, 0.34), (0.17, 0.63), (0.34, 1.26), (-0.46, 0.46), (-0.92, 0.92), (-0.63, -0.17), (-1.26, -0.34), (-0.17, -0.63), (-0.34, -1.26), (0.46, -0.46), (0.92, -0.92)]
pose existing nearby as 1 not creating new node
cur_ref_pose (0.63, 0.17) can be reached from  [0, 0, 0.0] with action 0 ?: True
for action 0 obs

### step 4

In [21]:
#STEP4 NO OB
action_step4 = 2
ob_id_step4 = 0 
excel_step = 0

#STEP4 WITH NEW OB
# action_step4 = 2
# ob_id_step4 = 4 
# excel_step = 4

pose_x = float(csvfile['Pose x for agent'].values[excel_step])
pose_y = float(csvfile['Pose Y for agent'].values[excel_step])
theta = float(csvfile['theta'].values[excel_step])
p_idx_step4 = ours.PoseMemory.pose_to_id([pose_x,pose_y], save_in_memory=False)
pose = ours.PoseMemory.id_to_pose(p_idx_step4)
odom_step4 = [pose[0],pose[1], theta]
obstacle_dists_step4= eval(csvfile['ob_dists'].values[excel_step])

In [22]:
ours.agent_manual_action_step(action_step4)
ours.agent_step_update(action_step4, [ob_id_step4,p_idx_step4,obstacle_dists_step4])

prior on believed state; action [2] 2 colour_ob: 0 inf pose: (0, 0) prior belief: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
update_B: a 2 qs[factor] [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] qs_prev[factor] [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
update_B: a 8 qs[factor] [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.] qs_prev[factor] [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Ghost nodes process:
______________________________
for action 0 obstacle 4.487496201197306 min_dist for new state 0.65
action, next pose and id 0 [0.63, 0.17] 1
next_pose [0.63, 0.17] [(0, 0), (0.63, 0.17), (1.26, 0.34), (0.17, 0.63), (0.34, 1.26), (-0.46, 0.46), (-0.92, 0.92), (-0.63, -0.17), (-1.26, -0.34), (-0.17, -0.63), (-0.34, -1.26), (0.46, -0.46), (0.92, -0.92)]
pose existing nearby as 1 not creating new node
cur_ref_pose (0.63, 0.17) can be reached from  [0, 0, 0.0] with action 0 ?: True
for action 0 obstacle 4.487496201197306 min_dist for new state 1.15
action, next pose and id 0 [1.26, 0.34] 2
next_pose [1.26, 0

# PLOT

In [ ]:
ours.agent_state_mapping

In [ ]:
print(ours.B[0].shape)
ours.agent_state_mapping

In [ ]:
ours.A[0][0]

In [ ]:
A0 = plot_likelihood(ours.A[0], ours.agent_state_mapping, tittle_add='observation')
A1 = plot_likelihood(ours.A[1], ours.agent_state_mapping, tittle_add='pose')

In [ ]:
plot = plot_state_in_map(ours.B[0], ours.agent_state_mapping)

In [ ]:
from aimapp.visualisation_tools import pickle_dump_model
from pathlib import Path

# pickle_dump_model(ours, store_path=Path('/home/idlab332/workspace/ros_ws/src/aimapp/aimapp'))

In [ ]:

pose_id = 0
visit = 0
from pathlib import Path
import os
a = str(pose_id) + '_' + str(visit)
store_path = Path.cwd() / 'tests' /  a

store_path
store_path = str(store_path)
os.path.exists(store_path)

In [ ]:
ours.B[0][:,0,10]

In [ ]:
B_plot = plot_transitions(ours.B[0], ours.agent_state_mapping, ours.possible_actions)

In [ ]:
B_plot = plot_transitions(ours.B[0], ours.agent_state_mapping, ours.possible_actions)

In [ ]:
B_plot_compare = compare_B1_B2_plots(B_v1_test1_ob, V_v1_test1_ob, ours.agent_state_mapping, ours.possible_actions)

# Random tests

In [ ]:
poses = ours.PoseMemory.poses
print(poses)
pose = poses[3]
for ref_p_idx, p in enumerate(poses):
    angle_p_to_pose = np.arctan2(pose[0], p[0]) - np.arctan2(pose[1], p[1])
    print(ref_p_idx, np.rad2deg(angle_p_to_pose))

    angle_p_to_pose = clip_rad_360(angle_p_to_pose)

    print(ref_p_idx, np.rad2deg(angle_p_to_pose))



In [ ]:
pose = (-0.63, 0.17)
p = [-0.63, -0.17]

euclidian_distance(pose, p)

Angles between poses

In [ ]:
pose = [0.56,0.15]
for ref_p_idx, p in enumerate(poses):
    # print(np.arctan2(pose[0], p[0]),np.arctan2(pose[1], p[1]))
    #angle between two points
    print('_')
    aangle_pose_to_p =np.arctan2(p[1]- pose[1], p[0]- pose[0])
    # aaangle_p_to_pose =clip_rad_360(np.arctan2(p[1]- 0, p[0]- 0))
    # print('angle_to_origin', np.rad2deg(aaangle_p_to_pose))
    aangle_pose_to_p = clip_rad_360(aangle_pose_to_p)
    #In reference to origin
    angle_pose_to_p = np.arctan2(p[1], p[0]) - np.arctan2(pose[1], pose[0])
#    angle_p_to_pose = np.arctan2(p[0], p[1]) - np.arctan2(pose[0], pose[1])

    angle_pose_to_p = clip_rad_360(angle_pose_to_p)
    dist_p_to_pose = euclidian_distance(pose, p)
    print(ref_p_idx, p,' to ', pose, 'angle', np.rad2deg(angle_pose_to_p), np.rad2deg(aangle_pose_to_p))


Point in polygons

In [ ]:
# Checking if a point is inside a polygon
def point_in_polygon(pt, polygon):
    num_vertices = len(polygon)
    inside = False
 
    # Store the first point in the polygon and initialize the second point
    p1 = polygon[0]
 
    # Loop through each edge in the polygon
    for i in range(1, num_vertices + 1):
        # Get the next point in the polygon
        p2 = polygon[i % num_vertices]
 
        # Check if the point is above the minimum y coordinate of the edge
        if pt[1] > min(p1[1], p2[1]):
            # Check if the point is below the maximum y coordinate of the edge
            if pt[1] <= max(p1[1], p2[1]):
                # Check if the point is to the left of the maximum x coordinate of the edge
                if pt[0] <= max(p1[0], p2[0]):
                    # Calculate the x-intersection of the line connecting the point to the edge
                    x_intersection = (pt[1] - p1[1]) * (p2[0] - p1[0]) / (p2[1] - p1[1]) + p1[0]
 
                    # Check if the point is on the same line as the edge or to the left of the x-intersection
                    if p1[0] == p2[0] or pt[0] <= x_intersection:
                        # Flip the inside flag
                        inside = not inside
 
        # Store the current point as the first point for the next iteration
        p1 = p2
 
    # Return the value of the inside flag
    return inside

POint in triangle

In [ ]:
import copy
def sign(p1, p2, p3):
    return (p1[0]- p3[0]) * (p2[1] - p3[1]) - (p2[0] - p3[0]) * (p1[1] - p3[1])

def point_in_triangle(pt, triangle_poses):
    p1,p2,p3 = triangle_poses[0], triangle_poses[1], triangle_poses[2]
    d1 = sign(pt, p1, p2)
    d2 = sign(pt, p2, p3)
    d3 = sign(pt, p3, p1)
    has_neg = (d1 < 0) or (d2 < 0) or (d3 < 0)
    has_pos = (d1 > 0) or (d2 > 0) or (d3 > 0) 
    return not (has_neg and has_pos)

odom = ours.PoseMemory.get_odom()
action_id = 0

zone_influence = ours.possible_actions[action_id][:]

triangle_poses = [odom]
print('odom', odom, 'IR', ours.influence_radius, 'ZI', zone_influence)
for angle_deg in zone_influence:
    angle_rad = np.deg2rad(angle_deg)
    x =2*ours.influence_radius * np.cos(angle_rad) + odom[0]
    y= 2*ours.influence_radius * np.sin(angle_rad) + odom[1]
    triangle_poses.append([x,y])
print('triangle_poses', triangle_poses)

point_in_triangle([0.5,0.25], triangle_poses)

# zone_influence = ours.possible_actions[action_id]
# max_dist_angle = (zone_influence[0] + zone_influence[1]) /2
# # not pretty but because Copy [:] etc is not really changing pointer...
# zone_influence = [zone_influence[0], max_dist_angle, zone_influence[1]]

# quadri_poses = [odom]
# print('odom', odom, 'IR', ours.influence_radius, 'ZI', zone_influence)
# for angle_deg in zone_influence:
#     angle_rad = np.deg2rad(angle_deg)
#     x =2*ours.influence_radius * np.cos(angle_rad) + odom[0]
#     y= 2*ours.influence_radius * np.sin(angle_rad) + odom[1]
#     quadri_poses.append([x,y])

# print(max_dist_angle,'quadri_poses', quadri_poses)

# point_in_polygon([0.5,0.25], quadri_poses)



In [ ]:
possible_actions = ours.possible_actions.copy()
possible_actions

In [ ]:
def pose_to_id(self, pose:list=None, odom:list=None, save_in_memory:bool = False) -> int:
        """
        return the position id from memory. 
        Evaluate if the given pose is in action range of other memorised poses 
        considering it's orientation difference and distance (<zone of influence)

        Parameters:
            pose (list): the pose we want to evaluate from memory
            save_in_memory (bool, optional): Do we want to save this pose in memory? 

        Returns:
            int: The pose id, -1 if none in memory and we don't want to remember this new pose
        """
        
        if pose is None:
            pose = self.odometry[:2]
        else:
            pose = pose[:2]

        if odom is None:
            odom = self.odometry[:2]
        else:
             odom = odom[:2]
        
        zone_action = self.possible_actions[0]
        p_idx = -1
        #TODO: CONSIDR ANGLE, ELSE WE Can'T HAVE 12 NODES IN SAME ZONE OF INF
        ref_closest_dist = self.influence_radius
        for ref_p_idx, p in enumerate(self.poses):
            print('_____')
            angle_pose_to_curr_pose = clip_rad_360(np.arctan2(pose[1]- odom[1], pose[0]- odom[0]))
            angle_p_to_curr_pose= clip_rad_360(np.arctan2(p[1]- odom[1], p[0]- odom[0])) 
            diff_angle =  abs(angle_pose_to_curr_pose -angle_p_to_curr_pose)
            # angle_ref_pose_to_origin = clip_rad_360(angle_ref_pose_to_origin)
            
            dist_p_to_pose = euclidian_distance(pose, p)
            print('ref idx and pose', ref_p_idx,p)
            print('angle pose to curr_pose', round(np.rad2deg(angle_pose_to_curr_pose),2), 'angle_p_to_curr_pose',round(np.rad2deg(angle_p_to_curr_pose),2),'dist', dist_p_to_pose)
            #If pose in the same action and influence zone
            #if abs(np.rad2deg(angle_p_to_pose)) <= np.mean(zone_action) and dist_p_to_pose < ref_closest_dist:
            if np.rad2deg(diff_angle) <= np.mean(zone_action) and dist_p_to_pose < ref_closest_dist:
                p_idx = ref_p_idx
                ref_closest_dist = dist_p_to_pose
                
        #No identified pose match current pose
        if p_idx < 0 and save_in_memory:
            pose = tuple(pose)
            self.poses.append(pose)
            p_idx = self.poses.index(pose)
        
        return p_idx

In [ ]:
def pose_transition_from_action(self,action:int, odom:list=None, ideal_dist:float=None):
        """
        Check if we have a pose in twice the area of the agen influence radius. 
        If we do, return this pose
        If we don't, we create a new node at a given ideal distance (if none given it's the influence radius + small value)
        """
        if odom is None:
            odom = self.odometry.copy()
        if self.possible_actions[action] == 'STAY':
            return odom
        
        zone_influence = self.possible_actions[action][:]
        triangle_poses = [odom]
        # print('odom', odom, 'IR', self.influence_radius, 'ZI', zone_influence)
        for angle_deg in zone_influence:
            angle_rad = np.deg2rad(angle_deg)
            x= 2*self.influence_radius * np.cos(angle_rad) + odom[0]
            y= 2*self.influence_radius * np.sin(angle_rad) + odom[1]
            triangle_poses.append([x,y])
        print('triangle_poses', triangle_poses)
        #check if pose in zone of action already exist in memory
        for pose in self.poses:
            pt_in_tr = point_in_triangle(pose[:2], triangle_poses)
            if pt_in_tr and not list(pose[:2]) == list(odom[:2]):
                return pose[:] #return corresponding pose
        if ideal_dist is None:
            ideal_dist = self.influence_radius + self.influence_radius/5 #just not be exactly on the radius
        average_angle = np.deg2rad(np.mean(zone_influence))
        x= ideal_dist * np.cos(average_angle) + odom[0]
        y= ideal_dist * np.sin(average_angle) + odom[1]
        return [x,y]

In [ ]:
triangles_poses = [[0.17, 0.97], [-0.3300000000000003, 0.10397459621556171], [0.16999999999999982, -0.030000000000000027]]
next_pose = [0.0, 0.0]
point_in_triangle(next_pose, triangle_poses)

In [ ]:
# next_pose = [0.97, 0.26]
# next_pose = [1.12, 0.0]
odom = [0.17, 0.97]
print(pose_transition_from_action(ours.PoseMemory, 8, odom=odom))
# print(pose_transition_from_action(ours.PoseMemory, 1, odom=odom))

# pose_to_id(ours.PoseMemory, next_pose, odom)

Point in triangle with arc

In [ ]:
def sign(p1, p2, p3):
    return (p1[0]- p3[0]) * (p2[1] - p3[1]) - (p2[0] - p3[0]) * (p1[1] - p3[1])

def is_clokwise_from_p1_to_p2(p1:list, p2:list)->bool:
    ''' 
    Determines whether `p2` is in the **clockwise direction** from `p1`.

    This is done by projecting `p2` onto the **perpendicular normal** of `p1`. 
    If the projection is **negative**, `p2` is clockwise from `p1`, otherwise counterclockwise.

    Parameters:
        p1 (list or tuple): The first 2D point `[x1, y1]`
        p2 (list or tuple): The second 2D point `[x2, y2]`

    Returns:
        bool: `True` if `p2` is **clockwise** from `p1`, otherwise `False`
    '''
    #p1_n= [-p1[1],p1[0]] #counter clockwise normal
    pt_proj = -p1[0]*p2[1] + p1[1]*p2[0]
    return pt_proj < 0

def is_within_radius_range(pt:list, arc_radius:float)->bool:
    '''
    Checks if a point `pt` lies within a circular region **centered at the origin** 
    with the given `arc_radius`.

    Parameters:
        pt (list or tuple): The 2D point `[x, y]`
        arc_radius (float): The radius of the circular region

    Returns:
        bool: `True` if `pt` is **inside or on** the circle, otherwise `False`
    '''
    return pt[0]**2 + pt[1]**2 <= arc_radius**2

def point_in_triangle_with_arc(pt:list, polygon:list)-> bool:
    '''
    Determines if a point `pt` lies **inside a triangular region with one curved edge**.

    The **polygon** is expected to have **4 points**:
    - `[odom, start_vector, end_vector, arc_center]`
    - The **arc** is formed between `start_vector` and `end_vector`, centered at `arc_center`.

    **Steps:**
    1. Check if the point is inside the regular **triangle** formed by `odom, start_vector, end_vector`.
    2. If not, check if it's inside the **curved region**:
        - The point must be within the **circular range**.
        - The point must be **between** `start_vector` and `end_vector` in a clockwise sense.

    Parameters:
        pt (list or tuple): The 2D point `[x, y]` to check.
        polygon (list of lists): A list of **4 points** `[p0, p1, p2, arc_center]`.

    Returns:
        bool: `True` if `pt` is inside the **triangle-with-arc**, otherwise `False`.
    
    NOTE: the odom point will not be considered in triangle with arc
    '''
    arc_center = polygon.pop(2)
    arc_radius = euclidian_distance(polygon[0], arc_center) #should be influence_radius*2
    p1, p2 = polygon[1], polygon[2]

    # Check if inside the triangle
    if point_in_triangle(pt, polygon):
        print('In triangle')
        return True 

    
    # If p2 vector is clocwise from pt and pt is clocwise from p1
    #  and in the circular region, then it's in the zone
    if is_within_radius_range(pt, arc_radius) and \
    is_clokwise_from_p1_to_p2(pt,p2) and not is_clokwise_from_p1_to_p2(pt,p1):
       return True

    return False 



In [ ]:
def quadrilater_points(self, action, odom):
    zone_influence = self.possible_actions[action][:]
    quadri_poses = [odom]
 
    max_dist_angle = (zone_influence[0] + zone_influence[1]) /2
    # not pretty but because Copy [:] etc is not really changing pointer...
    zone_influence = [zone_influence[0], max_dist_angle, zone_influence[1]]
    
    for angle_deg in zone_influence:
        angle_rad = np.deg2rad(angle_deg)
        x =2*self.influence_radius * np.cos(angle_rad) + odom[0]
        y= 2*self.influence_radius * np.sin(angle_rad) + odom[1]
        quadri_poses.append([x,y])

    return quadri_poses
        
def pose_transition_from_action(self,action:int, odom:list=None, ideal_dist:float=None):
    """
    Check if we have a pose in twice the area of the agen influence radius. 
    If we do, return this pose
    If we don't, we create a new node at a given ideal distance (if none given it's the influence radius + small value)
    """
    if odom is None:
        odom = self.odometry.copy()
    if self.possible_actions[action] == 'STAY':
        return odom
    
    
    polygon = quadrilater_points(self,odom)
    print('odom', odom, 'IR', self.influence_radius, 'ZI', zone_influence)
    
    #check if pose in zone of action already exist in memory
    for pose in self.poses:
        pt_in_tr = point_in_triangle(pose[:2], polygon)
        print('pt in polygon?', pt_in_tr)
        if pt_in_tr and not list(pose[:2]) == list(odom[:2]):
            return pose[:] #return corresponding pose
    if ideal_dist is None:
        ideal_dist = self.influence_radius + self.influence_radius/5 #just not be exactly on the radius
    average_angle = np.deg2rad(np.mean(polygon))
    x= ideal_dist * np.cos(average_angle) + odom[0]
    y= ideal_dist * np.sin(average_angle) + odom[1]
    return [x,y]


In [ ]:

# pop_elem = quadri.pop(2)

# euclidian_distance(pop_elem, odom)
# print(quadri, pop_elem)

check point assignement

In [ ]:
def point_in_triangle_with_arc(pt:list, quadri_poses:list)-> bool:
    '''
    Determines if a point `pt` lies **inside a triangular region with one curved edge**.

    The **polygon** is expected to have **4 points**:
    - `[odom, start_vector, end_vector, arc_center]`
    - The **arc** is formed between `start_vector` and `end_vector`, centered at `arc_center`.

    **Steps:**
    1. Check if the point is inside the regular **triangle** formed by `odom, start_vector, end_vector`.
    2. If not, check if it's inside the **curved region**:
        - The point must be within the **circular range**.
        - The point must be **between** `start_vector` and `end_vector` in a clockwise sense.

    Parameters:
        pt (list or tuple): The 2D point `[x, y]` to check.
        polygon (list of lists): A list of **4 points** `[p0, p1, p2, arc_center]`.

    Returns:
        bool: `True` if `pt` is inside the **triangle-with-arc**, otherwise `False`.
    
    '''
    polygon = quadri_poses[:]
    arc_point = polygon.pop(2)
    arc_radius = euclidian_distance(polygon[0], arc_point) #should be influence_radius*2
    p1, p2 = polygon[1], polygon[2]
    # print('polygon', polygon)
    # Check if inside the triangle
    if point_in_triangle(pt, polygon):
        # print(pt,'In triangle')
        # return True 
        pass

    print('<rad', is_within_radius_range(pt, polygon[0], arc_radius), ' pt to p2 clockwise',
    is_clokwise_from_p1_to_p2(pt,p2),  ' pt to p1 NOT clockwise', not is_clokwise_from_p1_to_p2(pt,p1))
    
    pt_between_vectors(pt, polygon[0], p1, p2)
    
    # If p2 vector is clocwise from pt and pt is clocwise from p1
    #  and in the circular region, then it's in the zone
    if is_within_radius_range(pt, polygon[0], arc_radius) :
       print(pt,'In arc')
    #    return True

    return False 

def pt_between_vectors(pt, odom, p1, p2):
    angle1 = clip_rad_360(np.arctan2(p1[1]-odom[1], p1[0]-odom[0]))
    angle2 = clip_rad_360(np.arctan2(p2[1]-odom[1], p2[0]-odom[0]))
    angle = clip_rad_360(np.arctan2(pt[1]-odom[1], pt[0]-odom[0]))
    return angle1 <= angle <= angle2
    # print(round(angle1,2) ,'<=', round(angle,2), '<=',round(angle2,2))
    # print( angle1 <= angle <= angle2)

In [ ]:
def pose_transition_from_actionn(self,action:int, odom:list=None, ideal_dist:float=None):
        """
        Check if we have a pose in twice the area of the agen influence radius. 
        If we do, return this pose
        If we don't, we create a new node at a given ideal distance (if none given it's the influence radius + small value)
        """
        if odom is None:
            odom = self.odometry.copy()
        if self.possible_actions[action] == 'STAY':
            return odom
        
        zone_influence = self.possible_actions[action][:]
        quadri = quadrilater_points(odom=odom, zone_influence=zone_influence,\
                                     influence_radius= self.influence_radius)
        
        print('quadri',quadri)
        x = []
        y = []
        for i in quadri:
             x.append(i[0])
             y.append(i[1])


        print(x, y)
        #check if pose in zone of action already exist in memory
        for i, pose in enumerate(self.poses):
            print('_') 
            print(i, 'pose',pose)
            pt_in_tr = point_in_triangle_with_arc(pose[:2], quadri)
            print('pt_in_tr', pt_in_tr)
            if pt_in_tr and not list(pose[:2]) == list(odom[:2]):
                return pose[:] #return corresponding pose
        print('no good point')
        if ideal_dist is None:
            ideal_dist = self.influence_radius + self.influence_radius/5 #just not be exactly on the radius
        average_angle = np.deg2rad(np.mean(zone_influence))
        x= ideal_dist * np.cos(average_angle) + odom[0]
        y= ideal_dist * np.sin(average_angle) + odom[1]
        return [x,y]

In [ ]:
def determine_next_pose(self, odom, action_id, min_dist_to_next_node):
        next_pose = pose_transition_from_actionn(self.PoseMemory, action =action_id, odom= odom, ideal_dist=min_dist_to_next_node)
        next_pose = [round(elem, 2) for elem in next_pose]
        next_pose_id = self.PoseMemory.pose_to_id(next_pose, save_in_memory=False)
        print('action, next pose and id', action_id, next_pose, next_pose_id)
        return next_pose, next_pose_id


In [ ]:
action_adjacent = 7
odom = [0.29, -1.09]
pt = [0.10,0.10]
next_adjacent_pose, next_adjacent_pose_id = determine_next_pose(ours, odom, \
                                                                action_adjacent, 0.65)

print(next_adjacent_pose)

# quadri = quadrilater_points(odom=odom, zone_influence=zone_influence,\
#                                      influence_radius= 0.5)
# quadri = quadrilater_points(ours.PoseMemory,  0, odom=odom)
# point_in_triangle_with_arc(pt, quadri)

In [ ]:
quadri = [[0.29, -1.09], [-0.5760254037844386, -1.5900000000000003], [-0.4171067811865477, -1.7971067811865475], [-0.21000000000000035, -1.9560254037844382]]
pt = (0.1, -1.4)
odom = (0.29, -1.09)

point_in_triangle_with_arc(pt, quadri)

# POLICIES GENERATION

In [ ]:
def get_quadrant_range(dx: int, dy: int) -> list:
    """Determine the quadrant range (angle interval) for reaching a goal pose."""
    if dx == 0 and dy == 0:
        raise ValueError("get_quadrant_range: dx and dy cannot both be zero.")

    # Convert angle to degrees in the range [0, 360)
    angle = np.degrees(np.arctan2(dy, dx)) % 360

    # Compute the quadrant using integer division
    quadrant = int(angle // 90)
    
    return [quadrant * 90, (quadrant + 1) * 90]

In [ ]:
def BFS_path_gen(dx:int, dy:int, moves:list):
    """ 
    based on the Breadth-First Search (BFS) we define the position motion considering all the possible moves.
    Given position x and y, the path ends when we reach either x or y with the moves, generating paths going in every directions     
    """
   # Initialize the queue with the starting point
    queue = deque([[(0, 0)]])
    
    valid_paths = []
    
    while queue:
        path = queue.popleft()
        x, y = path[-1]
        
        # Check all possible moves from the current position
        for move in moves:
            new_x, new_y = x + move[0], y + move[1]
            
            # Create a new path by extending the current path
            new_path = path + [(new_x, new_y)]
            
            # Check if the new position reaches the desired boundary
            if (abs(new_x) >= abs(dx) or abs(new_y) >= abs(dy)):
                valid_paths.append(new_path)
            # Only add the new path to the queue if it hasn't reached the boundary yet
            # Ensure this works for both positive and negative dx and dy
            elif (dx > 0 and new_x <= dx or dx < 0 and new_x >= dx) and (dy > 0 and new_y <= dy or dy < 0 and new_y >= dy):
                queue.append(new_path)
    
    return valid_paths

In [ ]:
def get_possible_poses_paths(start_pose:list, end_pose:list, actions:dict, influence_radius:float)-> list:
    ''' 
    given the start, end pose (as points) and possible motions (as orientations in degree), 
    define all possible paths with a BFS path generation.
    Return all possible path and motions
    '''
    dx,dy = int(end_pose[0] - start_pose[0]), int(end_pose[1] - start_pose[1]) # destination cell

    moves = {}
    print('start and end poses',start_pose, end_pose)
    #Get the angular direction the motion can have
    quadrant_range = get_quadrant_range(dx,dy)
    print('quadrant_range',quadrant_range)
    for angle_range in actions.values():
        if isinstance(angle_range, str):
            continue
        average_angle = np.mean(angle_range)
        angle_deg = float(average_angle) % 360
        
        #print('angle', angle_deg, angle_deg >= quadrant_range[0], angle_deg <= quadrant_range[1] )
        if (angle_deg >= quadrant_range[0] and angle_deg <= quadrant_range[1] ) \
            or angle_deg+360 == quadrant_range[1] :  #this is for 0==360
            x= influence_radius * np.cos(average_angle) + start_pose[0]
            y= influence_radius * np.sin(average_angle) + start_pose[1]
            # print(angle, motion)
            moves[(x,y)] = average_angle
    print('moves',moves)
    #If we want to explore, we want a grid path coverage (squared)
    paths = BFS_path_gen(dx, dy, list(moves.keys()))
    return paths, moves
current_pose = [0,0]
goal_poses = define_policies_objectives(current_pose, 3)
policies_lists = []
#get all the actions leading to the endpoints
actions = ours.possible_actions.copy()
endpoint = goal_poses[1]
#for endpoint in goal_poses:
paths, moves = get_possible_poses_paths(current_pose, endpoint, actions,influence_radius=1)
print(paths)
    # break

In [ ]:
import matplotlib.pyplot as plt

def plot_paths(paths):
    """Visualize multiple movement paths on a 2D plane."""
    plt.figure(figsize=(8, 6))
    
    # Offsetting factor to reduce overlap
    offset_step = 0.1  
    offsets = np.linspace(-offset_step, offset_step, len(paths))  

    for i, (path, offset) in enumerate(zip(paths, offsets)):
        x, y = zip(*path)  # Extract x and y coordinates
        
        # Apply slight offset to prevent excessive overlay
        x = np.array(x) + offset  
        y = np.array(y) + offset  
        
        plt.plot(x, y, marker='o', linestyle='--', linewidth=2, alpha=0.8, label=f'Path {i+1}')



    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.title('Movement Paths')
    plt.legend()
    plt.grid(True)
    plt.show()


plot_paths(paths)


In [ ]:
paths

In [ ]:
moves

In [ ]:
policies = create_policies(3, ours.possible_actions, 
        ours.current_pose, lookahead_distance=False,simple_paths= True)
len(policies)

In [ ]:
from aimapp.model.pymdp.maths import softmax, spm_dot, spm_log_single, softmax_obj_arr
from aimapp.model.pymdp import control
from aimapp.model.pymdp import utils


In [ ]:
def get_expected_states(qs, B, policy):
    """
    Compute the expected states under a policy, also known as the posterior predictive density over states

    Parameters
    ----------
    qs: ``numpy.ndarray`` of dtype object
        Marginal posterior beliefs over hidden states at a given timepoint.
    B: ``numpy.ndarray`` of dtype object
        Dynamics likelihood mapping or 'transition model', mapping from hidden states at ``t`` to hidden states at ``t+1``, given some control state ``u``.
        Each element ``B[f]`` of this object array stores a 3-D tensor for hidden state factor ``f``, whose entries ``B[f][s, v, u]`` store the probability
        of hidden state level ``s`` at the current time, given hidden state level ``v`` and action ``u`` at the previous time.
    policy: 2D ``numpy.ndarray``
        Array that stores actions entailed by a policy over time. Shape is ``(num_timesteps, num_factors)`` where ``num_timesteps`` is the temporal
        depth of the policy and ``num_factors`` is the number of control factors.

    Returns
    -------
    qs_pi: ``list`` of ``numpy.ndarray`` of dtype object
        Predictive posterior beliefs over hidden states expected under the policy, where ``qs_pi[t]`` stores the beliefs about
        hidden states expected under the policy at time ``t``
    """
    n_steps = policy.shape[0]
    n_factors = policy.shape[1]

    # initialise posterior predictive density as a list of beliefs over time, including current posterior beliefs about hidden states as the first element
    qs_pi = [qs] + [utils.obj_array(n_factors) for t in range(n_steps)]
    
    # get expected states over time
    for t in range(n_steps):
        for control_factor, action in enumerate(policy[t,:]):
            qs_pi[t+1][control_factor] = B[control_factor][:,:,int(action)].dot(qs_pi[t][control_factor])

    return qs_pi[1:]


In [ ]:
def calc_states_info_gain(A, qs_pi):
    """
    Computes the Bayesian surprise or information gain about states of a policy, 
    using the observation model and the hidden state distribution expected under that policy.

    Parameters
    ----------
    A: ``numpy.ndarray`` of dtype object
        Sensory likelihood mapping or 'observation model', mapping from hidden states to observations. Each element ``A[m]`` of
        stores an ``numpy.ndarray`` multidimensional array for observation modality ``m``, whose entries ``A[m][i, j, k, ...]`` store 
        the probability of observation level ``i`` given hidden state levels ``j, k, ...``
    qs_pi: ``list`` of ``numpy.ndarray`` of dtype object
        Predictive posterior beliefs over hidden states expected under the policy, where ``qs_pi[t]`` stores the beliefs about
        hidden states expected under the policy at time ``t``

    Returns
    -------
    states_surprise: float
        Bayesian surprise (about states) or salience expected under the policy in question
    """

    n_steps = len(qs_pi)

    states_surprise = 0
    for t in range(n_steps):
        states_surprise += control.spm_MDP_G(A, qs_pi[t])

    return states_surprise


In [ ]:
def get_expected_obs(qs_pi, A):
    """
    Compute the expected observations under a policy, also known as the posterior predictive density over observations

    Parameters
    ----------
    qs_pi: ``list`` of ``numpy.ndarray`` of dtype object
        Predictive posterior beliefs over hidden states expected under the policy, where ``qs_pi[t]`` stores the beliefs about
        hidden states expected under the policy at time ``t``
    A: ``numpy.ndarray`` of dtype object
        Sensory likelihood mapping or 'observation model', mapping from hidden states to observations. Each element ``A[m]`` of
        stores an ``numpy.ndarray`` multidimensional array for observation modality ``m``, whose entries ``A[m][i, j, k, ...]`` store 
        the probability of observation level ``i`` given hidden state levels ``j, k, ...``

    Returns
    -------
    qo_pi: ``list`` of ``numpy.ndarray`` of dtype object
        Predictive posterior beliefs over observations expected under the policy, where ``qo_pi[t]`` stores the beliefs about
        observations expected under the policy at time ``t``
    """

    n_steps = len(qs_pi) # each element of the list is the PPD at a different timestep

    # initialise expected observations
    qo_pi = []

    for t in range(n_steps):
        qo_pi_t = utils.obj_array(len(A))
        qo_pi.append(qo_pi_t)

    # compute expected observations over time
    for t in range(n_steps):
        for modality, A_m in enumerate(A):
            qo_pi[t][modality] = spm_dot(A_m, qs_pi[t])

    return qo_pi



In [ ]:
def calc_expected_utility(qo_pi, C):
    """
    Computes the expected utility of a policy, using the observation distribution expected under that policy and a prior preference vector.

    Parameters
    ----------
    qo_pi: ``list`` of ``numpy.ndarray`` of dtype object
        Predictive posterior beliefs over observations expected under the policy, where ``qo_pi[t]`` stores the beliefs about
        observations expected under the policy at time ``t``
    C: ``numpy.ndarray`` of dtype object
       Prior over observations or 'prior preferences', storing the "value" of each outcome in terms of relative log probabilities. 
       This is softmaxed to form a proper probability distribution before being used to compute the expected utility.

    Returns
    -------
    expected_util: float
        Utility (reward) expected under the policy in question
    """
    n_steps = len(qo_pi)
    
    # initialise expected utility
    expected_util = 0

    # loop over time points and modalities
    num_modalities = len(C)

    # reformat C to be tiled across timesteps, if it's not already
    modalities_to_tile = [modality_i for modality_i in range(num_modalities) if C[modality_i].ndim == 1]

    # make a deepcopy of C where it has been tiled across timesteps
    C_tiled = copy.deepcopy(C)
    for modality in modalities_to_tile:
        C_tiled[modality] = np.tile(C[modality][:,None], (1, n_steps) )
    
    C_prob = softmax_obj_arr(C_tiled) # convert relative log probabilities into proper probability distribution

    for t in range(n_steps):
        for modality in range(num_modalities):

            lnC = spm_log_single(C_prob[modality][:, t])
            expected_util += qo_pi[t][modality].dot(lnC)

    return expected_util

In [ ]:
ours.C

In [ ]:
tempo_C = ours.C[:]

tempo_C[1][1] = 5.0

tempo_C

In [ ]:
print(ours.possible_actions)
qs_pi = get_expected_states(ours.qs, ours.B, np.array([[12]]))
print(qs_pi)
qo_pi = get_expected_obs(qs_pi, ours.A)
print('qo', qo_pi[0])


ig = calc_states_info_gain(ours.A, qs_pi)
ut = calc_expected_utility(qo_pi, tempo_C)

print(ig, ut)
G = [-4.144062503084621, -4.194708019531143, -4.186214193646768, -2.0727028633817803, -4.324945251384786, -4.359559256543938, -4.718493266712995]
G = np.array(G)
lnE = spm_log_single(np.ones(G.shape) / len(G))
q = softmax(G)    
print(q.round(5))

q = softmax(G * 16)    
print(q.round(5))



In [ ]:
G = [-4.2, -4.5]
G = np.array(G)
print(np.ones(G.shape))
lnE = spm_log_single(np.ones(G.shape) / len(G))


In [ ]:
policies = [[[0]], [[2]], [[4]], [[6]], [[8]], [[10]], [[12]]]
print(policies) 
print(q_pi[0])
for pol_idx, policy in enumerate(policies):
    print('policy', policy)
    for factor_i, action_i in enumerate(policy[0, :]):
        print('h',factor_i, action_i, pol_idx)
        # q_pi[pol_idx]

In [ ]:
from aimapp.model.pymdp.control import sample_action
q_pi = []
for potential in q :
    q_pi.append(potential)
a= sample_action(q, policies = np.array([[[0]], [[12]]]), num_controls = ours.num_controls)
print(a, a[0], type(a[0]))

#### C pref

In [ ]:
def calc_expected_utility(qo_pi, C):
    """
    Computes the expected utility of a policy, using the observation distribution expected under that policy and a prior preference vector.

    Parameters
    ----------
    qo_pi: ``list`` of ``numpy.ndarray`` of dtype object
        Predictive posterior beliefs over observations expected under the policy, where ``qo_pi[t]`` stores the beliefs about
        observations expected under the policy at time ``t``
    C: ``numpy.ndarray`` of dtype object
       Prior over observations or 'prior preferences', storing the "value" of each outcome in terms of relative log probabilities. 
       This is softmaxed to form a proper probability distribution before being used to compute the expected utility.

    Returns
    -------
    expected_util: float
        Utility (reward) expected under the policy in question
    """
    n_steps = len(qo_pi)
    
    # initialise expected utility
    expected_util = 0

    # loop over time points and modalities
    num_modalities = len(C)

    # reformat C to be tiled across timesteps, if it's not already
    modalities_to_tile = [modality_i for modality_i in range(num_modalities) if C[modality_i].ndim == 1]

    # make a deepcopy of C where it has been tiled across timesteps
    C_tiled = copy.deepcopy(C)
    for modality in modalities_to_tile:
        C_tiled[modality] = np.tile(C[modality][:,None], (1, n_steps) )
    
    C_prob = softmax_obj_arr(C_tiled) # convert relative log probabilities into proper probability distribution
    print('C_prob', C_prob)
    for t in range(n_steps):
        for modality in range(num_modalities):

            lnC = spm_log_single(C_prob[modality][:, t])
            print('lnC', lnC)
            print('qo_pi[t][modality].dot(lnC)', qo_pi[t][modality].dot(lnC))
            expected_util += qo_pi[t][modality].dot(lnC)

    return expected_util

In [ ]:
ours.update_preference([0,-1], pref_weight=1)
ours.C

In [ ]:
prob_obs1 = [np.array([
    np.array([7.38229893e-01, 2.95937196e-04]),
    np.array([0.73793396, 0.00376346, 0.03764207, 0.03767897, 0.03771587,
              0.03775277, 0.03778967, 0.03782657, 0.03786347])
], dtype=object)]

prob_obs2 = [np.array([np.array([0.09313618, 0.00099094]),
       np.array([0.09214524, 0.00923136, 0.00932221, 0.00933144, 0.00934291,
              0.00934988, 0.83436671, 0.02009772, 0.00937573])           ],
      dtype=object)]
from aimapp.model.pymdp.maths import softmax, softmax_obj_arr, spm_dot, spm_wnorm, spm_MDP_G, spm_log_single

In [ ]:
u=calc_expected_utility(prob_obs1, ours.C)
print('prob_obs',u,prob_obs1)

In [ ]:
u = calc_expected_utility(prob_obs2, ours.C)
print('prob_obs',u, prob_obs2)